# **Educational Assistant Project: Leveraging Large Language Models in Education**

## **Project Overview**

### **Title**: Empowering Education in the Digital Age
### **Authors**: Bibi Nur Muhamad, Elijah Kulpinski, Fernando Vargas
### **Institution**: University of Wisconsin - Parkside, Kenosha WI USA
### **Date**: 12/04/23

#### **Abstract**
This project, titled "Empowering Education in the Digital Age," is a pioneering initiative aimed at enhancing the learning experience in computer science education through the use of Large Language Models (LLMs). Our goal is to develop an educational assistant that utilizes advanced AI techniques to provide personalized learning experiences, thereby improving student engagement and critical thinking skills.

#### **Objective**
The primary objective of this project is to explore the potential of LLMs in transforming educational methodologies. By integrating these models into educational content, we aim to create an interactive, adaptive learning environment that caters to the unique needs and learning styles of each student.

#### **Methodology**
The project encompasses a series of steps, beginning with the conversion of textbook content from PDF to XML format, followed by OCR processing of textbook datasets. We then transform Markdown content into XML for structured data representation. A significant part of our project involves generating conversation trees to simulate educational dialogues, which are vital for training our LLM. The conversation outputs in JSONL format are then converted to JSON for compatibility with various tools and platforms. The culmination of our project is the fine-tuning of the Mistral OpenOrca model in Google Colab, specifically tailored to our educational context.

#### **Expected Outcomes**
We anticipate that the integration of LLMs into educational content will result in a more engaging and effective learning experience. This project aims to demonstrate how AI can be used to create adaptive learning materials that respond to the needs of individual learners, thereby enhancing the overall quality of education.

#### **Scope of This Notebook**
This notebook serves as a comprehensive documentation and executable script of our entire research process. It is designed to ensure that our work is reproducible and well-documented. Each section of the notebook corresponds to a specific part of our project's workflow, complete with detailed explanations and code snippets.

---

### **Navigating the Notebook**:
1. **Textbook Conversion to XML**: Conversion of textbook data from PDF format to XML.
2. **OCR Textbook Dataset Processing**: Processing of the OCR dataset of textbook content.
3. **Markdown to XML Conversion**: Conversion of Markdown files into XML format.
4. **Synthetic Conversation Branch Generation**: Generation of conversation branches for training the LLM.
5. **JSONL to JSON Conversion**: Conversion of JSONL files to JSON format.
6. **Fine-Tuning Mistral OpenOrca in Google Colab**: Tailoring the language model to our educational context.
7. **[Experimental] Synthetic Conversation Tree Generation**: Generation of conversation trees that aren't 'good' or 'bad' but rather rated against the other possible responses in the tree for a relative rating to conversations in the tree. [Not Functioning Yet]

---

#### **Important Warning**:
This notebook is a merged version of multiple individual scripts. As such, no extensive testing has been done to evaluate how these scripts interact with one another within a single Colab instance. Users should be aware that unexpected behaviors might occur due to script interactions or environmental differences. It is recommended that users thoroughly test each section individually before executing the entire notebook in sequence.

#### **Additional Resources**:
- The detailed research paper associated with this project can be found at [GitHub: DividesZero](https://github.com/DividesZero).
- Trained models and datasets utilized in this project are available at [Hugging Face: ByteSized](https://huggingface.co/ByteSized).

#### **Note**:
This notebook is structured to be executed sequentially. Each section builds upon the previous one, ensuring a smooth flow and logical progression of the project. Users are encouraged to run the entire notebook in order to fully replicate our research findings.

In [ ]:
# ================================================
#    Pip Installs for OCR and Model Fine-Tuning
# ================================================

# Importing display module from IPython for clear output functionality
from IPython import display

# -------------------------------------
# Download Facebook's Nougat OCR Model
# -------------------------------------
# Facebook's Nougat is an OCR model that we use for text extraction from textbook images.
# This step installs the Nougat library from its GitHub repository.
!pip install git+https://github.com/facebookresearch/nougat
# Clearing the output to maintain a clean and readable notebook environment.
display.clear_output()

# -------------------------------------------------
# Download Fine-Tuning Libraries from Hugging Face
# -------------------------------------------------
# Hugging Face's 'transformers' library provides a collection of state-of-the-art
# pre-trained models for Natural Language Processing tasks.
# This command installs the library directly from its GitHub repository to ensure we have the latest version.
!pip install git+https://github.com/huggingface/transformers.git

# --------------------------------------------------------------------
# Install Additional Libraries for Model Acceleration and Fine-Tuning
# --------------------------------------------------------------------
# 'accelerate': Simplifies running transformers models on multi-GPU/multi-TPU setups.
# 'peft': Stands for PyTorch Efficient Finetuning, a library designed to accelerate fine-tuning.
# 'bitsandbytes': Optimizes memory and compute efficiency, particularly useful for large models.
# 'trl': Transformers Reinforcement Learning, used for training models via reinforcement learning.
# 'sentencepiece': A library for unsupervised text tokenization, crucial for handling diverse text inputs.
!pip install -q accelerate peft bitsandbytes transformers trl sentencepiece


In [ ]:
# ==============================================
#    Standard and Third Party Library Imports
# ==============================================

# ------------------------
# Standard Library Imports
# ------------------------
import asyncio          # Asynchronous I/O operations, useful for managing concurrent tasks.
import hashlib          # Hashing algorithms, used for hashing data                                         ##--Used only in Experimental Dataset Generation--##
import json             # Reading and writing JSON files, used for data serialization and deserialization.
import os               # Interfacing with the operating system, like handling file paths.
import random           # Generating random numbers, useful for stochastic processes or shuffling data.
import re               # Regular expressions, for text pattern matching.
import shutil           # High-level file operations, such as copying or moving files.
import subprocess       # Running subprocesses, useful for executing system commands.
import sys              # Access to system-specific parameters and functions.
import time             # Accessing time-related functions, like delays or timestamps.
import zipfile          # Working with ZIP archives, for compressing or extracting files.

# -------------------
# Third Party Imports
# -------------------
import matplotlib.pyplot as plt  # Plotting library for creating visualizations.
import numpy as np               # Fundamental package for scientific computing in Python.
import pandas as pd              # Data manipulation and analysis library.
import requests                  # Making HTTP requests to fetch data from the web.
import torch                     # PyTorch, a deep learning framework.
import xml.etree.ElementTree as ET  # Parsing and creating XML data.

from datasets import load_dataset, load_metric, Dataset  # Loading and handling datasets for machine learning.
from dotenv import load_dotenv  # Loading environment variables from a .env file.
from google.colab import drive  # Integrating Google Colab with Google Drive.
from huggingface_hub import snapshot_download, HfApi  # Interfacing with Hugging Face Hub.
from IPython import display  # Display tools for Jupyter notebooks.
from openai import AsyncOpenAI  # Asynchronous API for OpenAI services.
from pathlib import Path  # Object-oriented filesystem paths.
from peft import LoraConfig, PeftModel, get_peft_model  # Efficient fine-tuning of large models.
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator  # Accumulating data for TensorBoard.
from trl import SFTTrainer  # Supervised Fine-Tuning (SFT) training for transformer models.
from transformers import (  # Various utilities for working with transformer models.
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    DataCollatorForLanguageModeling, HfArgumentParser, LlamaTokenizerFast,
    TrainingArguments, TrainerCallback, pipeline)

# -----------------------------
# Google Colab Specific Imports
# -----------------------------
import google.colab.drive  # Specific to Google Colab, for drive integration.

# ------------
# PDF Handling
# ------------
import PyPDF2  # Library for PDF manipulation, like reading or splitting PDF files.

# ---------------
# Text Processing
# ---------------
import nltk  # Natural Language Toolkit, for processing and analyzing text.
import spacy # SpaCy, a library for advanced natural language processing.                                   ##--Used only in Experimental Dataset Generation--##
from textblob import TextBlob # TextBlob, a simple library for processing textual data.                     ##--Used only in Experimental Dataset Generation--##


---
# Key Variables and Configurations for Educational Assistant Project (`Educational_Assistant_Project.ipynb`)
---

In this cell, you will find key variables and configurations that can be adjusted to customize the behavior of the Educational Assistant Project script. These variables and configurations are organized into different sections based on their roles and functionalities. <br> <br>

## General Information

- This notebook serves as the main script for the Educational Assistant Project.
- It provides customizable variables and configurations to adapt the project to specific educational content and requirements. <br> <br>

## Section 1: Textbook Conversion to XML

This section contains variables and configurations related to the conversion of textbook data from PDF format to XML. <br> <br>

## Section 2: OCR Textbook Dataset Processing

Here, you will find variables and configurations for processing the OCR dataset of textbook content, including data cleaning and structuring. <br> <br>

## Section 3: Markdown to XML Conversion

This section focuses on variables and configurations for converting Markdown files to XML format, facilitating data structuring. <br> <br>

## Section 4: Conversation Branch Generation

In this section, you can customize variables and configurations for generating conversation branches for educational purposes. <br> <br>

## Section 5: JSONL to JSON Conversion

These variables are used for converting JSONL files to JSON format, potentially as part of data post-processing. <br> <br>

## Section 6: Fine-Tuning Large Language Model

This section provides variables relevant to fine-tuning a large language model, which can be important for specific project goals.

Please refer to each section to find specific variables and configurations that you can adjust according to your project's needs. <br> <br>

## Section 7: Conversation Tree Generation Experiment

These variables are used in the script intended to generate a synthetic dataset of conversation trees from an XML file of textbook paragraphs. <br> <br>


In [ ]:
# =====================================================================
#    Key Variables and Configurations for Textbook Conversion to XML
# =====================================================================

# Directory path for PDF files (specific to XML conversion)
XML_FOLDER_PATH = "xml_folder/path"

# Output file path for the combined XML dataset
XML_OUTPUT_FILE_PATH = os.path.join(XML_FOLDER_PATH, "combined_dataset.xml")

# Regular expression patterns for extracting metadata from textbooks
METADATA_PATTERNS = {
    'title_pattern': r'(?i)title[:\s]*([\w\s,]+)',
    'subtitle_pattern': r'(?i)subtitle[:\s]*([\w\s,]+)',
    'author_pattern': r'(?i)author[s]?[:\s]*([\w\s,]+)',
    'isbn_pattern': r'(?i)ISBN[-\s]*:?[\s]*([\d\-]+)',
    'edition_pattern': r'(?i)edition[:\s]*([\w\s,]+)',
    'publisher_pattern': r'(?i)publisher[:\s]*([\w\s,]+)',
    'publication_year_pattern': r'(?i)publication\s+year[:\s]*([\d]{4})',
    'chapter_pattern': r'Chapter\s*(\d+):\s*(.*)',
    'section_heading_pattern': r'Section:\s*(.*)',
    'page_number_pattern': r'Page\s*(\d+)',
    'references_pattern': r'References:\s*(.*)',
    'index_terms_pattern': r'Index Terms:\s*(.*)',
    'figures_pattern': r'Figure\s*(\d+):\s*(.*)',
    'footnotes_pattern': r'Footnote:\s*(.*)',
    'table_of_contents_pattern': r'(?:Contents|Table\s+of\s+Contents|Index)\s+(?:.\n)?\s(?:Chapter\s+1|Introduction)'
}

# Pattern for removing non-printable characters except for whitespace
NON_PRINTABLE_PATTERN = r'[^\x20-\x7E\n\r\t]'

# Pattern for detecting the start of a new paragraph
PARAGRAPH_START_PATTERN = r"^[A-Z]"

# NLTK Tokenizer Models
NLTK_MODELS = ['punkt']

# ===========================================================================
#    Key Variables and Configurations for OCR Textbook Dataset Processing
# ===========================================================================

# Google Drive paths
DRIVE_BASE_PATH = '/content/drive/My Drive/Colab Notebooks/Folder'
DATASET_ZIP_PATH = f'{DRIVE_BASE_PATH}/Dataset.zip'
DRIVE_MARKDOWN_DIR = f'{DRIVE_BASE_PATH}/Folder/'

# Paths in Google Colab
EXTRACTED_DATASET_PATH = '/content/Dataset'
LOCAL_MARKDOWN_DIR = '/content/OCR/'
LOCAL_TEXTBOOKS_DIR = '/content/Dataset/'
DRIVE_LOG_FILE = f'{DRIVE_BASE_PATH}/Markdown/nougat_ocr_log.txt'

# =====================================================================
#    Key Variables and Configurations for Markdown to XML Conversion
# =====================================================================

# Directory path for Markdown files
MARKDOWN_DIR = 'C:/Users/username/Desktop/Markdown'

# Output file path for the combined XML file
OUTPUT_XML_FILE = 'C:/Users/username/Desktop/Markdown/dataset.xml'

# Regular expression patterns for extracting metadata from textbooks
METADATA_PATTERNS = {
    'title_pattern': r'(?i)title[:\s]*([\w\s,]+)',
    'subtitle_pattern': r'(?i)subtitle[:\s]*([\w\s,]+)',
    'author_pattern': r'(?i)author[s]?[:\s]*([\w\s,]+)',
    'isbn_pattern': r'(?i)ISBN[-\s]*:?[\s]*([\d\-]+)',
    'edition_pattern': r'(?i)edition[:\s]*([\w\s,]+)',
    'publisher_pattern': r'(?i)publisher[:\s]*([\w\s,]+)',
    'publication_year_pattern': r'(?i)publication\s+year[:\s]*([\d]{4})',
    'chapter_pattern': r'Chapter\s*(\d+):\s*(.*)',
    'section_heading_pattern': r'Section:\s*(.*)',
    'page_number_pattern': r'Page\s*(\d+)',
    'references_pattern': r'References:\s*(.*)',
    'index_terms_pattern': r'Index Terms:\s*(.*)',
    'figures_pattern': r'Figure\s*(\d+):\s*(.*)',
    'footnotes_pattern': r'Footnote:\s*(.*)',
    'table_of_contents_pattern': r'(?:Contents|Table\s+of\s+Contents|Index)\s+(?:.\n)?\s(?:Chapter\s+1|Introduction)'
}

# =======================================================================
#    Key Variables and Configurations for Conversation Tree Generation
# =======================================================================

# File paths for input XML, output JSONL files, and console log
XML_INPUT_FILE_PATH = 'dataset.xml'
OUTPUT_JSONL_FILE_PATH = 'conversation_trees_dual.jsonl'
CONSOLE_LOG_FILE_PATH = 'console_log_dual.txt'

# OpenAI API key
API_KEY = "api-key-here"  # Replace with your actual API key

# Maximum number of paragraphs to process and the maximum depth of conversation trees
MAX_PARAGRAPHS = 2500   # Kept to 2500 Paragraphs To Keep API Costs < $20 (Cost about ~$15 For Our Dataset)
MAX_DEPTH = 5           # Depth of 5 x 2 Branch Types == 10 API Queries Per Paragraph

# Parameters for exponential backoff in case of rate limit errors
INITIAL_BACKOFF_DELAY = 1  # Initial delay in seconds
MAX_BACKOFF_DELAY = 60     # Maximum delay in seconds
MAX_RETRIES = 5

# Signals indicating the end of a conversation
END_SIGNALS = {"[TOPIC_END]", "[CONVERSATION_END]"}

# Filler content used in conversation trees
FILLER_CONTENT = {"[FILLER_CONTENT]", "[FAILED_TO_PARSE]"}

# Parameters for async processing
MAX_CONCURRENT_TASKS = 25  # Maximum number of concurrent tasks

# Output file path for post-processed and cleaned conversation trees
CLEANED_OUTPUT_JSONL_FILE_PATH = 'cleaned_conversation_trees_dual.jsonl'

# Model queried for dataset generation
OPENAI_API_MODEL = "gpt-3.5-turbo" # GPT 3.5 for Budget Constraints, GPT 4 for Quality

# ================================================
#    Key Variables for JSONL to JSON Conversion
# ================================================

# Path to the input JSONL file
JSONL_FILE_PATH = 'conversation_trees_dual_v3.jsonl'

# Path to the output JSON file
JSON_FILE_PATH = 'conversation_trees_dual_v3.json'

# ========================================================
#    Key Variables for Fine-Tuning Large Language Model
# ========================================================

# Model Information
model_name = "username/modelname"  # Replace with the base model from Hugging Face
dataset_name = "username/datasetname"  # Dataset for training
new_model = "new_modelname"  # Name for the fine-tuned model

# QLoRA Parameters
lora_r = 64  # LoRA attention dimension
lora_alpha = 16  # Alpha parameter for LoRA scaling
lora_dropout = 0.15  # Dropout probability for LoRA layers

# bitsandbytes Parameters
use_4bit = True  # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16"  # Compute dtype for 4-bit base models
bnb_4bit_quant_type = "nf4"  # Quantization type
use_nested_quant = False  # Nested quantization for 4-bit base models

# TrainingArguments Parameters
output_dir = "./results"  # Output directory for model predictions and checkpoints
num_train_epochs = 3  # Number of training epochs
fp16 = False  # Enable fp16 training
bf16 = False  # Enable bf16 training (use with A100)
per_device_train_batch_size = 2  # Training batch size per GPU
per_device_eval_batch_size = 2  # Evaluation batch size per GPU
gradient_accumulation_steps = 4  # Gradient accumulation steps
gradient_checkpointing = True  # Enable gradient checkpointing
max_grad_norm = 1.0  # Maximum gradient norm for clipping
learning_rate = 1e-4  # Initial learning rate
weight_decay = 0.01  # Weight decay for all layers except bias/LayerNorm
optim = "paged_adamw_32bit"  # Optimizer type
lr_scheduler_type = "cosine"  # Learning rate scheduler
max_steps = -1  # Maximum number of training steps
warmup_ratio = 0.06  # Ratio of steps for linear warmup
group_by_length = True  # Group sequences by length for efficiency
save_steps = 0  # Save checkpoint every X update steps
logging_steps = 1  # Log every X update steps

# SFT Parameters
max_seq_length = 896  # Maximum sequence length (reduce if VRAM limited)
packing = False  # Pack multiple short examples in the same sequence
device_map = {"": 0}  # Load model on specified GPU
neftune_noise_alpha = 5  # NEFTune noise alpha parameter

# Model Checkpoint and Configuration
final_epoch = "2.9995152690256908"  # Adjust based on the checkpoint name in your CoLab results folder
final_model_dir = os.path.join(output_dir, f"checkpoint-{final_epoch}")  # Directory where the final model is saved
pytorch_model_path = os.path.join(final_model_dir, 'pytorch_model.bin')  # Path to the saved pytorch_model.bin

# Model Loading Parameters
strict_model_loading = False  # Set to False to allow flexible loading of the model state dictionary

# Tokenizer Configuration
tokenizer_padding_side = "right"  # Define the padding side for the tokenizer
special_tokens_dict = {'additional_special_tokens': ['<GoodResponse>', '<BadResponse>', '<Prompt>', '<Response>']}  # Special tokens used in tokenizer

# Model Generation Configuration
model_config_use_cache = False  # Disable caching for stateful behavior in generation
model_config_pretraining_tp = 1  # Set tensor parallelism (relevant for distributed training)

# Generation Test Prompts
prompt1 = "Write a program for my school assignment that creates a JavaFX calculator."
prompt2 = "Solve the following integral: 4x^2+5x+2."

# Set to True to merge LoRA weights, False to keep them separate for use in multiple LoRas
merge_lora_weights = True

# Control whether to upload the model and tokenizer to Hugging Face
push_to_huggingface = True  # Set to True to upload, False to skip

# Hugging Face API token for manual authentication in the alternative method
hf_api_token = "your-token-here"  # Replace with your actual Hugging Face API token

# Model name to be used in Hugging Face Hub
new_model = "modelname"  # Replace with your desired model name

# Environment variables for the alternative upload method
hf_home_dir = '/content'  # Directory for Hugging Face configuration

# Model identifier on Hugging Face (username/modelname format)
hf_model_id = "username/modelname"  # Replace with your actual Hugging Face model identifier

# Directory to save the downloaded model
model_save_directory = "./"  # Adjust the directory as needed

# =================================================================
#    Key Variables for Experimental Conversation Tree Generation
# =================================================================
# Uncomment to for experimental tree generation
# API_KEY = "api-key-here" # TODO: Extract from environment variable
# MAX_DEPTH = 5
# MAX_BRANCHES = 16
# MAX_ATTEMPTS = 5
# MAX_DELAY = 60
# MAX_CONCURRENT_TASKS = 20
# INDIVIDUAL_PAIRS_FILE = 'individual_pairs.jsonl'
# RATED_TREES_FILE = 'rated_trees.jsonl'
# XML_FILE_PATH = 'output_short_snippet.xml'
# CONSOLE_LOG_FILE_PATH = 'console_log.txt'


---
# Textbook Conversion to XML (`PDF2XML.py`)
---

<a id='textbook-conversion-to-xml'></a>

## Description

The Python script named "PDF2XML.py" is used to convert textbook data from PDF format to XML. This script plays a crucial role in the initial data processing stage, setting the foundation for subsequent stages in the educational content pipeline.

### Purpose

The purpose of this script is to take educational content in PDF format and transform it into XML format. This conversion is a fundamental step in the fine-tuning pipeline, as XML is a structured and machine-readable format that is required for synethtic dataset generation.

In [ ]:
# Ensure NLTK tokenizer models are downloaded
for model in NLTK_MODELS:
    nltk.download(model, quiet=True)

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.

    Args:
        pdf_path (str): The file path of the PDF.

    Returns:
        str: Extracted and cleaned text from the PDF.
    """
    text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        # Iterating over each page in the PDF
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            # Extracting text from each page
            text += page.extract_text() if page.extract_text() else ''
    print(f"Text successfully extracted from {os.path.basename(pdf_path)}")

    # Remove non-printable characters except for whitespace characters
    cleaned_text = re.sub(NON_PRINTABLE_PATTERN, '', text)
    return cleaned_text

def extract_metadata(text):
    """
    Extracts metadata from the text using predefined patterns.

    Args:
        text (str): The text from which to extract metadata.

    Returns:
        dict: A dictionary of extracted metadata.
    """
    metadata = {}
    for key, pattern in METADATA_PATTERNS.items():
        match = re.search(pattern, text)
        # Assigning the extracted metadata or 'Unknown' if not found
        metadata[key] = match.group(1) if match else 'Unknown'
    return metadata

def extract_paragraphs(text):
    """
    Splits the text into paragraphs based on sentence tokenization.

    Args:
        text (str): The text to be split into paragraphs.

    Returns:
        list: A list of paragraphs.
    """
    sentences = nltk.tokenize.sent_tokenize(text)
    paragraphs = []
    paragraph = ""
    for sentence in sentences:
        # Start a new paragraph if the sentence begins with a capital letter
        if paragraph and re.match(PARAGRAPH_START_PATTERN, sentence):
            paragraphs.append(paragraph.strip())
            paragraph = sentence
        else:
            paragraph += " " + sentence
    # Add the last paragraph if it exists
    if paragraph:
        paragraphs.append(paragraph.strip())
    return paragraphs

def process_pdf_files(pdf_paths):
    """
    Processes multiple PDF files and converts them into an XML format.

    Args:
        pdf_paths (list): List of file paths for the PDFs to be processed.

    Returns:
        ElementTree.Element: The root element of the generated XML structure.
    """
    root = ET.Element("textbook_dataset")

    for pdf_path in pdf_paths:
        # Extracting text from the PDF
        text = extract_text_from_pdf(pdf_path)
        # Extracting metadata from the text
        metadata = extract_metadata(text)
        # Extracting paragraphs from the text
        paragraphs = extract_paragraphs(text)

        # Creating a new element for each book
        book_element = ET.SubElement(root, "book")
        # Adding metadata to the book element
        metadata_element = ET.SubElement(book_element, "metadata")
        for key, value in metadata.items():
            ET.SubElement(metadata_element, key).text = value

        # Adding each paragraph as a subelement of the book
        for paragraph in paragraphs:
            paragraph_element = ET.SubElement(book_element, "paragraph")
            paragraph_element.text = paragraph

    return root

def write_xml_to_file(root_element, output_file_path):
    """
    Writes an XML tree to a file.

    Args:
        root_element (ElementTree.Element): The root element of the XML tree.
        output_file_path (str): The file path to write the XML file.
    """
    tree = ET.ElementTree(root_element)
    with open(output_file_path, 'wb') as xml_file:
        # Writing the XML to a file with UTF-8 encoding
        tree.write(xml_file, encoding='utf-8', xml_declaration=True)

if __name__ == "__main__":
    # Gathering PDF paths from the specified folder
    pdf_paths = [os.path.join(XML_FOLDER_PATH, f) for f in os.listdir(XML_FOLDER_PATH) if f.endswith('.pdf')]
    # Processing each PDF and converting it into XML format
    combined_xml = process_pdf_files(pdf_paths)
    # Writing the combined XML data to a file
    write_xml_to_file(combined_xml, XML_OUTPUT_FILE_PATH)


---
# OCR Textbook Dataset Processing (`Nougat_OCR _Textbook_Dataset_V2.ipynb`)
---

## Description

This Jupyter Notebook, named "Nougat_OCR_Textbook_Dataset_V2.ipynb," is designed for processing an Optical Character Recognition (OCR) dataset containing textbook content. The notebook includes various data processing tasks, such as data cleaning, structuring, and preparation for model training and analysis.

### Note

- Always make a backup of your original OCR dataset before applying any data processing steps to avoid data loss.
- Immediately backup the resulting dataset upon conversion to aviod data loss incase the CoLab instance terminates.

In [ ]:
# -----------------------------------------------------------
# Mount Google Drive, Unzip Dataset, and Sync Markdown Files
# -----------------------------------------------------------
drive.mount('/content/drive')

# Check if the dataset directory already exists
if not os.path.exists(EXTRACTED_DATASET_PATH):
    # If not, unzip the dataset into the specified directory
    !unzip "$DATASET_ZIP_PATH" -d "$EXTRACTED_DATASET_PATH"
else:
    print("Dataset already extracted.")

# Ensure the local OCR directory exists
os.makedirs(LOCAL_MARKDOWN_DIR, exist_ok=True)

# Sync the Markdown files from Google Drive to the local OCR directory
!rsync -a --info=progress2 "$DRIVE_MARKDOWN_DIR" "$LOCAL_MARKDOWN_DIR"

# ------------------------------------------------------
# Process Textbooks with Nougat OCR and Update Log File
# ------------------------------------------------------
# Open the log file in append mode
with open(DRIVE_LOG_FILE, 'a') as log_file_writer:
    log_file_writer.write("Starting OCR process...\n")

    # Iterate over each file in the directory
    for file in os.listdir(LOCAL_TEXTBOOKS_DIR):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(LOCAL_TEXTBOOKS_DIR, file)
            markdown_file_name = os.path.splitext(file)[0] + ".mmd"
            local_markdown_file_path = os.path.join(LOCAL_MARKDOWN_DIR, markdown_file_name)

            # Check if the MMD file already exists locally
            if os.path.exists(local_markdown_file_path):
                print(f"Skipping '{file}' as MMD file already exists.")
                log_file_writer.write(f"Skipped processing of '{file}' as MMD file already exists.\n")
            else:
                print(f"Processing '{file}'...")
                # Process the PDF with Nougat OCR
                nougat_command = f'nougat --markdown pdf "{pdf_path}" --out "{LOCAL_MARKDOWN_DIR}"'
                os.system(f'{nougat_command} 2>&1 | tee -a "{DRIVE_LOG_FILE}"')

                # Check for the created Markdown file
                created_markdown_file_path = os.path.join(LOCAL_MARKDOWN_DIR, os.path.splitext(file)[0] + "_ocr.md")
                if os.path.exists(created_markdown_file_path):
                    # Copy the Markdown file to Google Drive
                    !cp "$created_markdown_file_path" "$DRIVE_MARKDOWN_DIR"
                    log_file_writer.write(f"Processed and saved Markdown for '{file}'.\n")
                else:
                    log_file_writer.write(f"Failed to process '{file}'.\n")


---
# Markdown to XML Conversion (`mmd_to_xml_script.py`)
---

## Description

This Python script is used to convert Markdown files into XML format. It facilitates data structuring and formatting for easier manipulation and analysis.

### Purpose

The purpose of this script is to streamline the process of converting Markdown files to XML, which can be particularly useful for data organization and analysis. The resulting XML format can be more structured and machine-readable, making it easier to work with and extract information from Markdown content.

### Usage

Before running the script, ensure you have Markdown files that you want to convert. You can execute this script with Python to perform the conversion. It is a helpful tool when you need Markdown content in a structured XML format for various applications, such as data analysis or integration with other systems.

In [ ]:
def process_metadata(line, parent_elem):
    """
    Extract metadata from a line and add it to the parent XML element.

    Args:
        line (str): The line of text to process.
        parent_elem (ElementTree.Element): The parent XML element to which the metadata will be added.

    Returns:
        bool: True if metadata was found and processed, False otherwise.
    """
    for key, pattern in METADATA_PATTERNS.items():
        match = re.match(pattern, line)
        if match:
            # Create a new sub-element for each metadata item found
            metadata_elem = ET.SubElement(parent_elem, key)
            metadata_elem.text = match.group(1)
            return True  # Metadata was processed
    return False  # No metadata was found

def mmd_to_xml(markdown_dir, output_xml_file):
    """
    Converts a directory of Markdown files into a single XML file.

    Args:
        markdown_dir (str): The directory containing Markdown files.
        output_xml_file (str): The path to the output XML file.

    Returns:
        str: A message indicating the completion and path of the generated XML file.
    """
    root = ET.Element("books")

    # Iterate over each Markdown file in the directory
    for file in os.listdir(markdown_dir):
        if file.endswith('.mmd'):
            mmd_file_path = os.path.join(markdown_dir, file)
            with open(mmd_file_path, 'r', encoding='utf-8') as mmd_file:
                lines = mmd_file.readlines()

            # Create a new book element for each Markdown file
            book = ET.SubElement(root, "book", name=file.replace('.mmd', ''))
            metadata = ET.SubElement(book, "metadata")
            content = ET.SubElement(book, "content")
            paragraph_lines = []  # Initialize paragraph lines for each book

            in_metadata = True  # Flag to track if currently processing metadata
            for line in lines:
                line = line.strip()
                # Skip lines that indicate missing or empty pages
                if line.startswith('[MISSING_PAGE_EMPTY:'):
                    continue

                # Process metadata lines at the beginning of the file
                if in_metadata and not process_metadata(line, metadata):
                    in_metadata = False  # End of metadata section

                # Process content after metadata
                if not in_metadata and line:
                    if line.startswith('* '):  # Treat bullet points as separate paragraphs
                        paragraph = ET.SubElement(content, "paragraph")
                        paragraph.text = line[2:]  # Exclude the bullet point marker
                    else:
                        # Aggregate lines into paragraphs, separate by empty lines
                        if not line or line == os.linesep:
                            if paragraph_lines:
                                paragraph = ET.SubElement(content, "paragraph")
                                paragraph.text = ' '.join(paragraph_lines)
                                paragraph_lines.clear()
                        else:
                            paragraph_lines.append(line)

            # Handle the final paragraph if it exists
            if paragraph_lines:
                paragraph = ET.SubElement(content, "paragraph")
                paragraph.text = ' '.join(paragraph_lines)

    # Create and write the XML tree to the specified file
    tree = ET.ElementTree(root)
    tree.write(output_xml_file, encoding='utf-8', xml_declaration=True)

    return f'Combined XML file generated at: {output_xml_file}'

# Running the conversion process
result = mmd_to_xml(MARKDOWN_DIR, OUTPUT_XML_FILE)
print(result)


---
# Script 4: Conversation Branch Generation (`generate-conversation-tree-async-dual.py`)
---

**Author**: Elijah Kulpinski <br>
**Date**: 12/04/23 <br>
**Version**: 2.0.1 <br>

## Description

This Python script is designed to generate conversation trees for educational purposes, focusing on topics in computer science and education. The script operates asynchronously, enabling parallel generation of multiple conversation trees. Each tree is derived from a paragraph extracted from a collection of textbook content. The primary objective is to foster critical thinking and a deeper understanding of the subject matter in students. The trees simulate educational dialogues, serving as a crucial part of the data preparation stage for training language models.

## Key Features

- **Asynchronous Processing:** The script generates multiple conversation trees simultaneously, optimizing resource usage and reducing processing time. This feature is particularly important when dealing with a large number of paragraphs, ensuring scalability and efficiency.

- **Contextual Tree Generation:** Each conversation tree is built incrementally, rooted in a specific paragraph from a textbook. This approach guarantees that each tree is contextually relevant and coherent, directly tied to the educational content.

- **Branching Logic:** The script employs a dynamic branching logic to expand conversations. This logic takes into account the content and educational goals of each paragraph, generating conversations that are not only relevant but also pedagogically sound.

- **Quality Control:** The script includes mechanisms for automated quality checks, ensuring consistency and coherence in the generated conversations. Manual oversight is also incorporated for educational relevance, allowing for fine-tuning and adjustments to align with specific teaching objectives.

- **API Rate Limit Management:** The script is designed to monitor and manage API usage efficiently, staying within the constraints of rate limits. This ensures smooth operation and minimizes the risk of service interruptions.

## Usage

Before running the script, ensure all dependencies are installed and that the necessary API keys are set up. The script can be executed with Python 3.x. Each run processes a set of paragraphs from the XML file, generating a unique conversation tree for each paragraph. The output is a set of JSONL files, each representing a conversation tree.

## Note

Access to the OpenAI API is required for this script. It is crucial to adhere to the API's rate limits and usage guidelines. Ensure you have the necessary permissions and API quota to use this script effectively. The script is designed with flexibility in mind, allowing users to modify parameters such as the number of paragraphs processed and the depth of conversation trees, to suit different requirements and constraints.

In [ ]:
# ============================================================
#    Functions for Synthetic Conversation Branch Generation
# ============================================================

def parse_xml(file_path, max_paragraphs=MAX_PARAGRAPHS):
    """
    Parses an XML file and extracts paragraphs along with their book titles.

    Args:
        file_path (str): The file path to the XML file.
        max_paragraphs (int): The maximum number of paragraphs to extract.

    Returns:
        list: A list of tuples, each containing a book title and a paragraph.
    """
    book_paragraphs = []
    book_counts = {}
    total_paragraphs = 0

    try:
        # Parsing the XML file
        parser = ET.XMLParser(encoding="utf-8")
        tree = ET.parse(file_path, parser=parser)
        root = tree.getroot()

        # Counting paragraphs in each book
        for book in root.findall('book'):
            book_title = book.get('name')
            paragraphs = book.findall('.//paragraph')
            book_counts[book_title] = len(paragraphs)
            total_paragraphs += len(paragraphs)

        # Allocating paragraphs proportionally to each book
        for book in root.findall('book'):
            book_title = book.get('name')
            percentage = book_counts[book_title] / total_paragraphs
            allocated_paragraphs = int(percentage * max_paragraphs)
            paragraphs = book.findall('.//paragraph')
            selected_paragraphs = random.sample(paragraphs, min(allocated_paragraphs, len(paragraphs)))

            # Extracting text from each paragraph
            for paragraph in selected_paragraphs:
                text = paragraph.text.strip() if paragraph.text else ''
                book_paragraphs.append((book_title, text))

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except ET.ParseError:
        print(f"Error: The file '{file_path}' is not a valid XML or is malformed.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return book_paragraphs

def append_to_file(file_path, data):
    """
    Appends the given data to a file.

    Args:
        file_path (str): The path to the file where data will be appended.
        data (dict): The data to append.
    """
    with open(file_path, 'a') as file:
        # Dumping data as JSON into the file
        json.dump(data, file)
        file.write('\n')  # Adding a newline for separation

def backoff_hdlr(attempt, delay=INITIAL_BACKOFF_DELAY):
    """
    Implements an exponential backoff strategy to handle rate limit errors.

    Args:
        attempt (int): The current retry attempt number.
        delay (int): The delay in seconds before the next retry. Defaults to INITIAL_BACKOFF_DELAY.

    Effects:
        Induces a sleep period with a delay that exponentially increases with each attempt.
    """
    # Calculate the delay with jitter to avoid synchronized retries
    jitter = random.uniform(0, attempt)
    new_delay = min(delay * 2, MAX_BACKOFF_DELAY) + jitter  # Cap the delay at MAX_BACKOFF_DELAY
    print(f"Rate limit hit, backing off for {new_delay} seconds for attempt {attempt}.")
    with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
        log_file.write(f"Rate limit hit, backing off for {new_delay} seconds for attempt {attempt}.\n")
    time.sleep(new_delay)

def parse_response(response_text):
    """
    Parses the response text from the API to extract the student's question and the educator's response.

    Args:
        response_text (str): The response text from the API.

    Returns:
        tuple: A tuple containing the student's question and the educator's response. If parsing fails, both are set to "FAILED_TO_PARSE".
    """
    try:
        # Attempt to parse the response text as JSON
        response_json = json.loads(response_text)
        with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
            log_file.write(f"Parsing Response Text: \n{response_text}\n")

        # Extract the 'Prompt' and 'Response' fields from the JSON
        student_question = response_json.get("Prompt", "FAILED_TO_PARSE")
        educator_response = response_json.get("Response", "FAILED_TO_PARSE")

        return student_question, educator_response

    except json.JSONDecodeError:
        # Handle cases where the response text is not valid JSON
        with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
            log_file.write("Failed to parse response as JSON.\n")
        return "FAILED_TO_PARSE", "FAILED_TO_PARSE"

# Async function to generate conversation tree with book title and controlled concurrency
async def generate_conversation_branch(client, book_title, paragraph, branch_type, semaphore, index):

    """
    Asynchronously generates a single branch of a conversation tree for a given paragraph from a textbook, including the
    book title for context. It can generate either a "good" or "bad" conversation branch based on the provided branch_type parameter.

    Args:
    client (AsyncOpenAI): The OpenAI API client for making asynchronous requests.
    book_title (str): The title of the book to which the paragraph belongs.
    paragraph (str): The paragraph to base the conversation tree on.
    branch_type (str): The type of branch to generate, either 'good' or 'bad'.
    semaphore (asyncio.Semaphore): Semaphore to control concurrency.
    index (int): The current paragraph index.

    Returns:
    dict: The generated conversation branch as a dictionary, including the book title, paragraph index, and the conversation steps.
    Usage:
    """

    async with semaphore: # Control concurrency with semaphore
        # Initial conversation tree structure
        start_time = time.perf_counter()  # Define start_time at the beginning of the block
        conversation_tree = {"book_title": book_title, "paragraph_index": index, "conversation": []}
        max_retries = 5
        conversation_summary = ""
        depth = 0

        while depth < MAX_DEPTH:
            attempt = 0
            delay = 1

            # Generate prompt for the conversation step
            if branch_type == 'good':
                if depth == 0:
                    # Root Node's Instructions for Good Branch
                    prompt = (
                    "Assume the role of an educator assisting a student. You are here to help them develop critical thinking skills and engage with the content."
                    "Based on the concept in the provided paragraph from the book titled: '{}', "
                    "generate a prompt-response pair in JSON for use in a conversation tree in a dataset to fine-tune an LLM. "
                    "Begin from the student's point-of-view with a question reflecting a student's inquiry or assigned task regarding the textbook paragraph "
                    "contents, then follow up from the educator's point-of-view with a response that encourages critical thinking "
                    "and that engages the student. Your response should offer guidance, examples, or suggestions, helping the student explore the concept "
                    "further, while maintaining a professional tone. Avoid directly solving the problem; instead, focus on guiding the student to find the "
                    "solution themselves. If the conversation reaches a natural conclusion or if the topic changes significantly, include "
                    "'[CONVERSATION_END]' or '[TOPIC_END]' respectively.\nHere's the textbook paragraph that was assigned to the student: {}"
                    "Don't forget you are to generate both a Prompt and a Response in JSON to represent the start of the conversation between the student and educator."
                    "Example: You receive an assigned textbook paragraph from a math textbook on solving basic integrals for calculus 1 students. "
                    "Generated Output should be like: 'Prompt: Complete this integral: 2x^2 + 2. Response: I can't solve it for you but I can help guide you "
                    "in solving it for yourself. How many steps have you gotten in solving the integral so far? "
                    "(Hint: Start by trying to break up this one big integral into multiple smaller, easier-to-solve integrals)'. "
                ).format(book_title, paragraph[:500])

                elif depth >= MAX_DEPTH - 3 and depth < MAX_DEPTH + 1:
                    # Leaves Instructions for Good Branch
                    prompt = (
                    "Assume the role of an educator assisting a student, and are currently in conversation with the student. "
                    "Based on the concept in the provided paragraph from the book titled: '{}', your goal is to start concluding the "
                    "conversation effectively without directly completing the students assignment so they can develop critical thinking skills "
                    "and engage with their course content. Your response should offer guidance, examples, or suggestions, helping the student "
                    "explore the concept further, while maintaining a professional tone. Avoid directly solving the problem; instead, focus on "
                    "guiding the student to find the solution themselves. Your job is to craft a prompt-response pair in JSON, aiming to gracefully "
                    "conclude the interaction. Ensure your response helps in summarizing key points or providing final thoughts, without introducing "
                    "new complex topics. Include '[CONVERSATION_END]' or '[TOPIC_END]' if appropriate. "
                    "\nAssigned Textbook Paragraph Contents: {}\nPrior Conversation Summary: {}\n"
                    "Don't forget you are to generate both a Prompt and a Response in JSON to represent the conversation approaching its conclusion between the student and educator."
                    "Example 1: 'Prompt: Wow, thanks! I'm feeling more confident for my exam! Response: You're welcome, the key is breaking down complex integrals into simpler parts. Always feel free to revisit our discussion on this topic. Good luck on your exam! [CONVERSATION_END]'"
                    "Example 2: 'Prompt: I'm pretty confident on integrals, time to start English homework, can you assist with that? Response: I'm glad, if you ever want more practice on integrals just ask away. Let me know if I can guide you for any other school content. [TOPIC_END]'"
                ).format(book_title, paragraph[:500], conversation_summary)

                else:
                    # Child Node Instructions for Good Branch
                    prompt = (
                        "Assume the role of an educator assisting a student, and are currently in conversation with the student. "
                        "Based on the concept in the provided paragraph from the book titled: '{}', your goal is to continue guiding the student towards learning "
                        "the content they need to answer their question in their inital prompt; without directly completing the students assignment outright, in order for "
                        "them to develop critical thinking skills and engage with their course content. Your response should offer guidance, examples, "
                        "or suggestions, helping the student explore the concept further, while maintaining a professional tone. Avoid directly solving "
                        "the problem; instead, focus on guiding the student to find the solution themselves. Your job is to craft a prompt-response pair "
                        "in JSON. If the conversation reaches a natural conclusion or if the topic changes significantly, include '[CONVERSATION_END]' or '[TOPIC_END]' respectively. "
                        "Assigned Textbook Paragraph Content: {}\nPrior Conversation Summary: {}\n"
                        "Don't forget you are to generate both a Prompt and a Response in JSON to represent the progression of the conversation between the student and educator."
                        "Example 1: 'Prompt: I never really understood how variables work in programming? Response: Think of variables as containers for storing data. What data types do you think can be stored in these containers?'"
                        "Example 2: 'Prompt: I don't really know the difference between Comparator and Comparable in Java? Response: To understand the differences between Comparator and Comparable in Java, consider the purpose and "
                        "usage of each: Comparable is for natural ordering within a class (using compareTo), whereas Comparator allows for custom ordering outside a class (using compare). Reflect on their design implications, "
                        "flexibility, and specific use cases to grasp why and when each interface is used in Java programming.'"
                    ).format(book_title, paragraph[:500], conversation_summary)

            elif branch_type == 'bad':
                if depth == 0:
                    # Root Node's Instructions for Bad Branch
                    prompt = (
                        "Assume the role of an educator, tasked with responding to a student based on the concept from the assigned paragraph in the book titled: '{}'. "
                        "Create a prompt-response pair in JSON where the response directly answers the student's inquiry about the textbook paragraph contents, "
                        "Your response should direct and simply answer the student's question about the textbook content, discouraging further exploration or critical thinking from the student. "
                        "Begin with generating a student's question related to the paragraphs content as the Prompt and provide the educator's response that is concise and avoids depth as the Response."
                        "This approach, while not ideal, is to illustrate ineffective teaching in a dataset so the better you demonstrate what not to do via the conversation, the higher "
                        "quality the dataset is to learn from.\n"
                        "Include '[CONVERSATION_END]' or '[TOPIC_END]' if applicable. \nAssigned Textbook Paragraph: {}\n"
                        "Don't forget you are to generate both a Prompt and a Response in JSON to represent the start of the ineffective conversation between the student and educator."
                        "Example: 'Prompt: How do I solve this integral: 2x^2 + 2? Response: The answer is 2/3x^3 + 2x. This is a trivial integral you could've just entered into a graphing calculator for the answer.'"
                    ).format(book_title, paragraph[:500])

                elif depth >= MAX_DEPTH - 3 and depth < MAX_DEPTH + 1:
                    # Leaves Instructions for Bad Branch
                    prompt = (
                        "Assume the role of an educator assisting a student, and are currently in conversation with the student. "
                        "Based on the concept in the provided paragraph from the book titled: '{}', your goal is to start concluding the conversation while demonstrating ineffective teaching methods/communication. "
                        "The response should be straightforward, providing minimal guidance or encouragement for critical thinking. "
                        "This method illustrates a less effective approach to teaching. When crafting the response, it is that it is more about giving information than fostering understanding.\n"
                        "The worse you are at being an educator, the better the dataset works at showing what not to do. Keep the tone professional but avoid encouraging student exploration.\n"
                        "Assigned Textbook Content: {}\nPrior Conversation Summary: {}\nInclude '[CONVERSATION_END]' or '[TOPIC_END]' if applicable."
                        "Don't forget you are to generate both a Prompt and a Response in JSON to represent the conversations abrupt conclusion between the student and educator."
                        "Example 1: 'Prompt: What's the importance of this scientific concept? Response: It's just something you need to know for the test. Don’t overthink it. Office hours are ending, I've got to end this conversation. [CONVERSATION_END]'"
                        "Example 2: 'Prompt: I'm bored, this reading doesn't make sense. Response: Yeah its just something you gotta do, the real world is much harder kid.'"
                    ).format(book_title, paragraph[:500], conversation_summary)

                else:
                    # Child Node Instructions for Bad Branch
                    prompt = (
                        "Assume the role of an educator assisting a student, and are currently in conversation with the student. "
                        "Based on the concept in the provided paragraph from the book titled: '{}', your goal is to generate a conversation of an educator responding to their student while demonstrating ineffective teaching methods/communication, "
                        "without a thorough summary or invitation for further thought. This may not stop the student from asking further questions, but the educator should keep demonstrating ineffective instruction methods."
                        "The response should be quick and dismissive, demonstrating a suboptimal way of concluding an educational interaction.\nThe reason you are doing this is for creating a dataset of what not to do so it's perfectly "
                        "The worse you are at being an educator, the better the dataset works at teaching what not to do."
                        "Textbook Paragraph Content: {}\nPrior Conversation Summary: {}\nInclude '[CONVERSATION_END]' or '[TOPIC_END]' if applicable."
                        "Don't forget you are to generate both a Prompt and a Response in JSON to quickly end or negatively influence the conversation between the student and educator."
                        "Example 1: 'Prompt: I'm still not clear about this topic. Response: Its not a big deal. Just focus on the main points, no need to understand everything.'"
                        "Example 2: 'Prompt: Im struggling to understand recursion in programming. Can you explain it more clearly? Response: Recursion is just a specific complex loop. Dont get too hung up on it. Anyway, we need to move on to the next topic.'"
                    ).format(book_title, paragraph[:500], conversation_summary)

            else:
                # Log an error if the branch type is not recognized
                print(f"Branch type neither good nor bad, an error occurred somewhere for {index} ('{book_title}') at Depth: {depth}.")
                with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                    log_file.write(f"Branch type error for {index} ('{book_title}') at Depth: {depth}.\n")

            try:
                # Track the start time of the API call for performance monitoring
                api_call_start_time = time.perf_counter()
                print(f"Paragraph {index} ('{book_title}'): Generating Prompt-Response (Depth: {depth}). Start Time: {api_call_start_time:.2f}")
                with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                    log_file.write(f"API call start for Paragraph {index} ('{book_title}'), Depth: {depth}.\n")

                # Make an asynchronous API call to generate the conversation step
                response = await client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "system", "content": prompt}]
                )

                # Calculate and log the time taken for the API call
                api_call_end_time = time.perf_counter()
                print(f"API response received for Paragraph {index} ('{book_title}'). Duration: {api_call_end_time - api_call_start_time:.2f} seconds.")
                with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                    log_file.write(f"API response duration for Paragraph {index}: {api_call_end_time - api_call_start_time:.2f} seconds.\n")

                # Process the API response if it contains choices
                if response.choices:
                    user_query = response.choices[0].message.content
                    student_question, educator_response = parse_response(user_query)

                    # Construct a conversation entry with relevant details
                    entry = {
                        "book_title": book_title,
                        "paragraph_index": index,
                        "depth": depth,
                        "response_type": branch_type,
                        "response": user_query
                    }
                    conversation_tree["conversation"].append(entry)
                    append_to_file(output_file_path, entry)

                    # Log the generated conversation entry
                    with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                        log_file.write(f"Generated conversation entry: {entry}\n")

                    depth += 1  # Increment the depth for the next iteration

                    # Check for end signals to terminate the conversation
                    if any(signal in user_query for signal in END_SIGNALS):
                        print(f"Conversation ended for Paragraph {index} at Depth: {depth}.")
                        with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                            log_file.write(f"Conversation end signal detected for Paragraph {index} at Depth: {depth}.\n")
                        break

                    # Generate a summary prompt for the next step of the conversation
                    summary_prompt = (
                        "Summarize the following conversation in a single paragraph. '{}':\n{}"
                    ).format(book_title, conversation_summary + " " + user_query)
                    print(f"Generating summary for Paragraph {index} at Depth: {depth}.")
                    with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                        log_file.write(f"Summary generation start for Paragraph {index} at Depth: {depth}.\n")

                    # Make an API call to generate the conversation summary
                    summary_response = await client.chat.completions.create(
                        model=OPENAI_API_MODEL,
                        messages=[{"role": "system", "content": summary_prompt}]
                    )

                    # Process the summary response
                    if summary_response.choices:
                        conversation_summary = summary_response.choices[0].message.content
                    else:
                        print(f"No summary generated for Paragraph {index}.")
                        with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                            log_file.write(f"No summary generated for Paragraph {index}.\n")
                        break
                else:
                    print(f"No response generated for Paragraph {index}.")
                    with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                        log_file.write(f"No response for Paragraph {index}.\n")
                    break

            except Exception as e:
                # Handle exceptions, especially rate limit errors with exponential backoff
                error_message = str(e).lower()
                if 'rate limit' in error_message:
                    attempt += 1
                    backoff_hdlr(attempt, delay)
                else:
                    print(f"Error generating conversation for Paragraph {index}: {e}")
                    with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                        log_file.write(f"Error for Paragraph {index}: {e}\n")
                    break

            if attempt >= max_retries:
                print(f"Max retries reached for Paragraph {index}. Moving to next paragraph.")
                with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
                    log_file.write(f"Max retries reached for Paragraph {index}.\n")
                break

        end_time = time.perf_counter()
        print(f"Finished processing Paragraph {index} in total time: {end_time - start_time:.2f} seconds.")
        with open(CONSOLE_LOG_FILE_PATH, 'a') as log_file:
            log_file.write(f"Total processing time for Paragraph {index}: {end_time - start_time:.2f} seconds.\n")
        append_to_file(OUTPUT_JSONL_FILE_PATH, conversation_tree)

    return conversation_tree

# Async Function to Generate a Conversation Tree
async def generate_conversation_tree(client, semaphore, book_title, paragraph, index, total, OUTPUT_JSONL_FILE_PATH, CONSOLE_LOG_FILE_PATH):
    '''
    Orchestrates the generation of a complete conversation tree for a given paragraph by creating both 'good' and 'bad' branches.

    Args:
        client (AsyncOpenAI): OpenAI API client for asynchronous requests.
        semaphore (asyncio.Semaphore): Semaphore to control the level of concurrency.
        book_title (str): The title of the textbook.
        paragraph (str): The specific paragraph from the textbook.
        index (int): The index of the current paragraph.
        total (int): Total number of paragraphs to process.
        OUTPUT_JSONL_FILE_PATH (str): Path to the output JSONL file.
        CONSOLE_LOG_FILE_PATH (str): Path to the console log file.

    Returns:
        dict: A complete conversation tree containing both 'good' and 'bad' branches.
    '''

    print(f"Generating 'good' branch for paragraph {index} ('{book_title}').")
    # Generate the 'good' branch of the conversation
    good_branch = await generate_conversation_branch(client, book_title, paragraph, 'good', semaphore, index)

    print(f"Generating 'bad' branch for paragraph {index} ('{book_title}').")
    # Generate the 'bad' branch of the conversation
    bad_branch = await generate_conversation_branch(client, book_title, paragraph, 'bad', semaphore, index)

    # Combining both branches into a complete conversation tree
    conversation_tree = {
        "book_title": book_title,
        "paragraph_index": index,
        "good_branch": good_branch,
        "bad_branch": bad_branch
    }
    return conversation_tree

# Async Function to Process All Paragraphs
async def process_paragraphs_async(book_paragraphs, OUTPUT_JSONL_FILE_PATH, console_log_file_path):
    '''
    Asynchronously processes each paragraph from the textbook to generate conversation trees.

    Args:
        book_paragraphs (list): List of paragraphs from the textbook.
        OUTPUT_JSONL_FILE_PATH (str): Path to the output JSONL file.
        console_log_file_path (str): Path to the console log file.

    '''

    # Initialize the AsyncOpenAI client
    async_client = AsyncOpenAI(api_key=api_key)
    # Control concurrency with a semaphore
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_TASKS)
    tasks = []

    # Loop through each paragraph and generate conversation trees
    try:
        for index, (book_title, paragraph) in enumerate(book_paragraphs):
            print(f"Processing paragraph {index + 1}/{len(book_paragraphs)} from '{book_title}'.")
            # Create a task for each paragraph
            task = generate_conversation_tree(async_client, semaphore, book_title, paragraph, index, len(book_paragraphs), OUTPUT_JSONL_FILE_PATH, console_log_file_path)
            tasks.append(task)

        # Run all tasks concurrently
        await asyncio.gather(*tasks)

    except Exception as e:
        print(f"Error encountered during processing: {e}")

    # Introduce a delay to manage API rate limits
    time.sleep(0.5)

# Function to Post-Process Conversation Trees
def post_process_conversation_trees(input_file_path, output_file_path, signals, filler_content):
    '''
    Cleans and refines the conversation trees, removing unnecessary content and formatting the data.

    Args:
        input_file_path (str): Path to the input file containing raw conversation trees.
        output_file_path (str): Path to the output file for cleaned conversation trees.
        signals (set): Set of signals indicating the end of a conversation.
        filler_content (set): Set of filler content used in conversation trees.
    '''

    processed_trees = []
    # Read each line in the input file and process the conversation trees
    with open(input_file_path, 'r') as file:
        for line in file:
            tree = json.loads(line)
            processed_conversation = []

            if 'conversation' not in tree:
                continue  # Skip invalid entries

            for entry in tree["conversation"]:
                raw_response = entry.get("api_response", "")
                student_question, educator_response = parse_response(raw_response)

                new_entry = {
                    "book_title": tree["book_title"],
                    "paragraph_index": tree["paragraph_index"],
                    "depth": entry["depth"],
                    "response_type": entry["response_type"],
                    "student_question": student_question,
                    "educator_response": educator_response
                }

                if student_question in filler_content or educator_response in filler_content:
                    continue  # Skip entries with filler content

                for signal in signals:
                    new_entry["student_question"] = new_entry["student_question"].replace(signal, '')
                    new_entry["educator_response"] = new_entry["educator_response"].replace(signal, '')

                processed_conversation.append(new_entry)

            tree["conversation"] = processed_conversation
            processed_trees.append(tree)

    # Write the processed trees to the output file
    with open(output_file_path, 'w') as file:
        for tree in processed_trees:
            json.dump(tree, file)
            file.write('\n')

# Main Execution Block
if __name__ == "__main__":
    # Check for file existence and initialize processing
    if not os.path.exists(OUTPUT_JSONL_FILE_PATH):
        with open(OUTPUT_JSONL_FILE_PATH, 'w'): pass
    if not os.path.exists(CONSOLE_LOG_FILE_PATH):
        with open(CONSOLE_LOG_FILE_PATH, 'w'): pass

    print("Starting the Conversation Tree Generation script.")
    book_paragraphs = parse_xml(XML_INPUT_FILE_PATH)
    asyncio.run(process_paragraphs_async(book_paragraphs, OUTPUT_JSONL_FILE_PATH, CONSOLE_LOG_FILE_PATH))

    print("Post-processing conversation trees.")
    post_process_conversation_trees(OUTPUT_JSONL_FILE_PATH, CLEANED_OUTPUT_FILE_PATH, END_SIGNALS, FILLER_CONTENT)

    print("Conversation tree generation completed.")


---
# JSONL to JSON Conversion (`jsonl-to-json.py`)
---
## Description
This script is designed to convert JSONL (JSON Lines) files into standard JSON format. Such a conversion is crucial for ensuring data uniformity and compatibility with various processing and analysis tools that require or perform better with standard JSON structures.

In [ ]:
# This script reads each line of the JSONL file, converts it to a JSON object,
# and aggregates these objects into a list. The list is then written to a standard
# JSON file with proper formatting. This process is essential for converting line-delimited
# JSON data into a more universally accepted JSON format.

# Initialize an empty list to store JSON objects
json_data = []

# Read the JSONL file
with open(JSONL_FILE_PATH, 'r') as file:
    # Read the file line by line
    lines = file.read().split('\n')
    for line in lines:
        if line:  # Ensure the line is not empty
            # Parse the JSON line and add it to the list
            json_data.append(json.loads(line))

# Write the JSON data to a file
with open(JSON_FILE_PATH, 'w') as file:
    # Dump the JSON data with an indentation of 4 spaces for readability
    json.dump(json_data, file, indent=4)


---
# Fine-Tuning Large Language Model (`fine_tune_mistral_openorca_in_google_colab_v2.ipynb`)
---

**Authors:** Elijah Kulpinski and Fernando Vargas  
**Last Update:** 12/04/23

### Description
This script is dedicated to fine-tuning the Mistral-OpenOrca Large Language Model (LLM) in a Google Colab environment. It is designed for advanced model customization, enabling researchers and developers to tailor LLMs to specific tasks or datasets.

### Acknowledgements
- Original concept by [@maximelabonne](https://twitter.com/maximelabonne).
- Based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da).
- Credits to Tolga HOŞGÖR for his VRAM optimization solution.

### Runtime Environment
This script is optimized for A100 GPU usage. For VRAM limitations, adjust the batch size or max_seq_length.

### Usage Instructions
1. Specify the model information using the username/modelname from HuggingFace.
2. Create a HuggingFace API Key with Read/Write permissions and input it in the designated cell.
3. Choose 'n' for Git credential setup during the process.
4. Optional GGUF conversion: Post fine-tuning, convert the model to GGUF format on a CPU instance for cost efficiency.

### Important Notes
- This script is part of a larger project aimed at enhancing the capabilities of LLMs in educational and research settings.
- Monitor resource usage, especially VRAM, and adjust settings as needed.

### References for Additional Context and Further Reading
- Fine-tuning Llama2: [Llama2 Fine-tuning Colab Notebook](https://colab.research.google.com/drive/1PEQyJO1-f6j0S_XJ8DV50NkpzasXkrzd?usp=sharing)
- Fine-tuning Mistral: [Mistral Fine-tuning Colab Notebook](https://colab.research.google.com/drive/17L3hoNGtCdgRTq4JU4koKHF6gvQmv9Pd?usp=sharing)
- Mistral Inference: [Mistral Inference Colab Notebook](https://colab.research.google.com/drive/1yZlLSifCGELAX5GN582kZypHCv0uJuNX?usp=sharing)

In [ ]:
###########################
# Dataset Preparation
###########################

# Initialize the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=False)
# 'add_eos_token=True' adds an end-of-sentence token, which is necessary for certain model types.
# 'use_fast=False' ensures compatibility across various tokenizers but may be slower.

# Add custom special tokens to the tokenizer and resize the model's token embeddings
special_tokens_dict = {'additional_special_tokens': ['<GoodResponse>', '<BadResponse>']}
tokenizer.add_special_tokens(special_tokens_dict)
# These special tokens can be used to indicate different types of responses in the training data.

# Define a function to format a batch of samples
def format_custom(samples, tokenizer, include_special_tokens=True):
    formatted_samples = []  # List to hold formatted samples

    # Iterate over each sample and format it
    for i in range(len(samples['response_type'])):
        # Select the appropriate response type token
        response_type_token = "<GoodResponse>" if samples['response_type'][i] == 'good' else "<BadResponse>"
        prompt_parts = [f"{response_type_token}\n"] if include_special_tokens else []

        # Format each part of the sample
        for key in samples:
            value = samples[key][i]
            part = f"{value}\n\n" if key in ['prompt', 'response'] else f"{key}: {value}\n\n"
            prompt_parts.append(part)

        # Join the parts and add to the list
        formatted_samples.append("".join(prompt_parts).strip() + tokenizer.eos_token)

    return formatted_samples

# Define a function to apply the formatting template to a batch
def template_batch(batch, tokenizer, include_special_tokens=True):
    batch["text"] = format_custom(batch, tokenizer, include_special_tokens)
    return {"text": batch["text"]}

# Load the dataset from Hugging Face datasets
full_dataset = load_dataset(dataset_name)

# Select the 'train' split if available, otherwise use the full dataset
dataset = full_dataset['train'] if 'train' in full_dataset else full_dataset

# Split the dataset into training and evaluation sets (70% train, 30% test)
train_test_split = dataset.train_test_split(test_size=0.3)
training_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Apply the formatting function to the training dataset
# Including special tokens for differentiating response types
training_dataset = training_dataset.map(
    lambda batch: template_batch(batch, tokenizer, include_special_tokens=True),
    batched=True,
    remove_columns=training_dataset.column_names
)

# Apply the formatting function to the evaluation dataset
# Without special tokens to assess model performance in a more realistic setting
eval_dataset = eval_dataset.map(
    lambda batch: template_batch(batch, tokenizer, include_special_tokens=False),
    batched=True,
    remove_columns=eval_dataset.column_names
)

#############################
# Tokenizer and Model Setup
#############################

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Configure bitsandbytes for model optimization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,  # Enable 4-bit model loading
    bnb_4bit_quant_type=bnb_4bit_quant_type,  # Define the quantization type
    bnb_4bit_compute_dtype=compute_dtype,  # Set the compute data type
    bnb_4bit_use_double_quant=use_nested_quant,  # Enable nested quantization if required
)

# Check GPU compatibility with bfloat16 (bfloat16 is beneficial for newer GPUs like A100)
# This block dynamically adjusts settings based on GPU capability
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:  # Check for GPU architecture compatibility (major version 8 or above)
        bf16 = True  # Use bfloat16 if compatible (beneficial for high VRAM GPUs)
        print("=" * 80)
        print("Your GPU supports bfloat16: setting bf16=True")
        print("=" * 80)

# Load Mistral-OpenOrca tokenizer with custom special tokens
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=False)
special_tokens_dict = {'additional_special_tokens': ['<GoodResponse>', '<BadResponse>', '<Prompt>', '<Response>']}
tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to end-of-sequence token
tokenizer.padding_side = "right"  # Define padding side (right padding)

# Load the base model with the bitsandbytes configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Apply bitsandbytes configuration
    device_map=device_map           # Define device mapping (GPU distribution)
)

# Resize model token embeddings to align with the tokenizer's vocabulary size
model.resize_token_embeddings(len(tokenizer))

# Disable caching to prevent reusing past key values for attention
model.config.use_cache = False

# Set pretraining tensor parallelism (relevant for distributed training environments)
model.config.pretraining_tp = 1

##########################
# Training Configuration
##########################

# Initialize and start Tensorboard for Training Logging
%load_ext tensorboard
%tensorboard --logdir results/runs
# These commands load and start Tensorboard within the notebook, allowing real-time monitoring of training metrics.

# Define a callback class for saving model checkpoints at the end of each epoch
class SaveCheckpointCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Create a directory for the current epoch's checkpoint
        output_dir = os.path.join(args.output_dir, f"checkpoint-{state.epoch}")
        os.makedirs(output_dir, exist_ok=True)
        # Save the model state and tokenizer to the directory
        torch.save(model.state_dict(), os.path.join(output_dir, 'pytorch_model.bin'))
        tokenizer.save_pretrained(output_dir)

# Data collator to handle variable length batches efficiently on GPU
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Masked Language Modeling is set to False for Causal Language Modeling
    pad_to_multiple_of=8  # Padding to a multiple of 8 optimizes GPU memory utilization
)

# Function to compute evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute and return accuracy and perplexity metrics
    accuracy = load_metric("accuracy")
    accuracy_result = accuracy.compute(predictions=predictions, references=labels)
    loss = torch.nn.CrossEntropyLoss()
    perplexity = torch.exp(loss(logits, torch.tensor(labels))).item()
    return {"accuracy": accuracy_result["accuracy"], "perplexity": perplexity}

# Load QLoRA configuration for the model
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    inference_mode=False,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]  # Target specific layers for LoRA modifications
    # Additional layers can be included for deeper fine-tuning but may impact performance
)

# Set training parameters for the Hugging Face Trainer
training_arguments = TrainingArguments(
    output_dir=output_dir,  # Directory for saving model outputs and checkpoints
    num_train_epochs=num_train_epochs,  # Total number of training epochs
    per_device_train_batch_size=per_device_train_batch_size,  # Batch size per GPU for training
    gradient_accumulation_steps=gradient_accumulation_steps,  # Number of steps for gradient accumulation
    optim=optim,  # Optimizer type
    save_steps=save_steps,  # Frequency of checkpoint saving
    logging_steps=logging_steps,  # Logging frequency
    learning_rate=learning_rate,  # Learning rate
    weight_decay=weight_decay,  # Weight decay for regularization
    fp16=fp16,  # Use fp16 precision
    bf16=bf16,  # Use bf16 precision if applicable
    max_grad_norm=max_grad_norm,  # Gradient clipping
    max_steps=max_steps,  # Maximum number of training steps
    warmup_ratio=warmup_ratio,  # Warmup ratio for learning rate scheduling
    group_by_length=group_by_length,  # Group sequences by length for efficiency
    lr_scheduler_type=lr_scheduler_type,  # Type of learning rate scheduler
    report_to="tensorboard"  # Log metrics to Tensorboard
)

# Initialize the trainer for supervised fine-tuning
trainer = SFTTrainer(
    model=model,  # The model to be trained
    train_dataset=training_dataset,  # Training dataset
    eval_dataset=eval_dataset,  # Evaluation dataset
    compute_metrics=compute_metrics,  # Function to compute metrics
    data_collator=data_collator,  # Data collator
    peft_config=peft_config,  # LoRA configuration
    dataset_text_field="text",  # Field in dataset containing the text
    max_seq_length=max_seq_length,  # Maximum sequence length
    tokenizer=tokenizer,  # Tokenizer
    args=training_arguments,  # Training arguments
    packing=packing,  # Whether to pack multiple examples
    neftune_noise_alpha=neftune_noise_alpha,  # NEFTune noise alpha parameter
    callbacks=[SaveCheckpointCallback()]  # Callbacks for additional functionality
)

##########################
# Training Model
##########################
# Start training the model
try:
    trainer.train()
    # If the script crashes due to VRAM limitations, consider reducing 'per_device_train_batch_size' or 'max_seq_length'.
    # You can also increase 'gradient_accumulation_steps' to effectively increase batch size without increasing VRAM usage.
except RuntimeError as e:
    if "out of memory" in str(e):
        print("ERROR: The model training ran out of memory.")
        print("TIPS: Reduce the batch size or sequence length. Alternatively, consider using gradient accumulation to handle larger batches.")
    else:
        raise e  # Re-raise the exception for any other errors

# Save the trained model to the specified directory
trainer.model.save_pretrained(new_model)
# Ensure that you have sufficient disk space to save the model.
# If the saving process fails, check the available disk space or try saving to a different location.

# Note: After training, it's beneficial to evaluate the model on a validation set or with specific benchmarks to assess its performance.

#########################
# Text Generation Pipeline
#########################

# Load the trained model and tokenizer for text generation

# Load the model's state_dict saved at the final epoch
model_state_dict = torch.load(pytorch_model_path)
# This loads the saved model weights from the specified path

# Initialize the model using the base model name
model = AutoModelForCausalLM.from_pretrained(model_name)
# Here, the base pre-trained model is loaded to ensure compatibility with the trained weights

# Resize the model's token embeddings to accommodate special tokens added during training
model.resize_token_embeddings(len(tokenizer))
# This step is crucial to match the tokenizer's size with the model's embeddings

# Update model configuration based on the requirements of the generation task
model.config.use_cache = model_config_use_cache
model.config.pretraining_tp = model_config_pretraining_tp
# These configurations are set to optimize the model's performance during text generation

# Load the saved state dictionary into the model with flexible matching
try:
    model.load_state_dict(model_state_dict, strict=strict_model_loading)
    # 'strict=False' allows loading the model even if some keys in the state dictionary don't match perfectly
    print("\nModel's state dictionary loaded successfully.")
except RuntimeError as e:
    # Catch and report any errors during the loading process
    print("\nError occurred while loading the model's state dictionary:", e)

# Reload the tokenizer and incorporate any special tokens used during training
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = tokenizer_padding_side
# The tokenizer is reinitialized to ensure it aligns with the model's training configuration

# Initialize the text generation pipeline with the fine-tuned model and tokenizer
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
# The pipeline facilitates easy text generation with the model and tokenizer

# Generate responses for the given prompts
result1 = pipe(f"<s>[INST] {prompt1} [/INST]")
result2 = pipe(f"<s>[INST] {prompt2} [/INST]")
# The prompts are formatted with special tokens (if used during training) to guide the model's response generation

# Print the generated text for each prompt
print("\nGenerated Text for Prompt 1:")
print(result1[0]['generated_text'])

print("\nGenerated Text for Prompt 2:")
print(result2[0]['generated_text'])
# The results are displayed to evaluate the model's performance on the specific tasks

# Note: If the generated text is not satisfactory, consider revising the model's training data or further fine-tuning.
# Ensure that the prompts align with the type of content the model was trained on.

# ==================================
# Clearing VRAM to Free Up Resources
# ==================================

# Empty VRAM
# When working with large models like LLMs, it's common to face VRAM (Video RAM) limitations.
# This snippet ensures that all large variables, especially those related to the model, are deleted from memory,
# allowing for the VRAM to be freed up. This step is crucial in environments like Google Colab,
# where VRAM is a limited resource.

# Delete the model, pipeline, and trainer objects
del model   # Delete the model instance to free up VRAM
del pipe    # Delete the pipeline instance
del trainer # Delete the trainer instance

# Invoke garbage collection
# Python's garbage collector (gc) is used to free up memory. Although Python automatically manages memory,
# explicitly calling the garbage collector ensures that all unused objects are promptly cleared,
# especially in a notebook environment where variables can persist in memory across different cells.
import gc   # Import the garbage collection module
gc.collect()  # Run garbage collection once
gc.collect()  # Run it a second time to catch any remaining unreferenced objects

# After running this snippet, a significant amount of VRAM should be freed.
# This is especially useful before starting a new task that requires substantial memory,
# such as loading another large model or processing a large dataset.

##################################
# Reload Model with LoRA weights
##################################

# This section is designed to reload the model with LoRA weights. The inclusion of the 'merge_lora_weights' key variable allows users to decide whether to merge these weights into the base model.

# Check if LoRA weights should be merged
if merge_lora_weights:
    print("Merging LoRA weights.")
    # Load the base model in FP16 precision if merging is desired
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        low_cpu_mem_usage=True,
        return_dict=True,
        torch_dtype=torch.float16,  # Set the data type to FP16 for efficiency
        device_map=device_map,      # Device mapping for distributed training
    )

    # Merge the base model with the trained LoRA weights
    model = PeftModel.from_pretrained(base_model, new_model)
    model = model.merge_and_unload()
    # 'merge_and_unload()' combines the LoRA layers with the base model and optimizes memory usage
else:
    # Load the base model without merging LoRA weights
    print("Not merging LoRA weights, assigning 'model' to base model.")
    model = AutoModelForCausalLM.from_pretrained(model_name)
    # This approach is useful when users want to maintain separate LoRA weights

# Reload the tokenizer and add special tokens
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Resize the model's token embeddings to accommodate special tokens
model.resize_token_embeddings(len(tokenizer))
# Ensuring that the tokenizer and model embeddings are synchronized, especially with added tokens

# The model is now reloaded with or without merged LoRA weights, depending on the user's choice set in 'merge_lora_weights'.

#########################
# Push to HuggingFace
#########################
if push_to_huggingface:
    try:
        # Attempt to login to Hugging Face using the CLI
        get_ipython().system('huggingface-cli login')  # This will prompt for the token
        print("Model and tokenizer are being uploaded to Hugging Face.")

        # Push the model to Hugging Face
        model.push_to_hub(new_model, use_temp_dir=False)

        # Push the tokenizer to the same Hugging Face repository
        tokenizer.push_to_hub(new_model, use_temp_dir=False)

        print("Upload successful. Model and tokenizer are now available on Hugging Face Model Hub.")

    except Exception as primary_exception:
        print(f"An error occurred during Hugging Face upload: {primary_exception}")
        print("Attempting alternative upload method.")

        # Set environment variables for the alternative method
        os.environ['HF_HOME'] = hf_home_dir
        os.environ['HF_API_TOKEN'] = hf_api_token
        os.environ['token'] = hf_api_token

        # Fallback to the alternative method if the primary method fails
        try:
            model.push_to_hub(new_model, use_temp_dir=False)
            tokenizer.push_to_hub(new_model, use_temp_dir=False)
            print("Upload successful using the alternative method.")
        except Exception as e:
            print(f"An error occurred during the alternative Hugging Face upload: {e}")
else:
    print("Model and tokenizer are not being uploaded to Hugging Face.")

#################################################################
# LlamaCPP HF Conversion to GGUF (Recommended on a CPU Instance)
#################################################################
# It's recommended to switch to a CPU instance for this part of the process to save costs,
# as GGUF conversion does not require GPU acceleration.

# Function to download the model from Hugging Face
def download_model_from_hf(model_id, save_directory):
    # Download the model snapshot from Hugging Face Hub
    return snapshot_download(repo_id=model_id, cache_dir=save_directory)

# Function to convert the model to GGUF format
def convert_to_gguf(model_name, model_directory):
    # Clone LlamaCPP and install dependencies if not already present
    if not os.path.exists('llama.cpp'):
        subprocess.run(["git", "clone", "https://github.com/ggerganov/llama.cpp.git"])
        subprocess.run(["pip", "install", "-r", "llama.cpp/requirements.txt"])

    gguf_outfile_name = os.path.join(model_directory, f"{model_name}.gguf")

    # Run the conversion script to GGUF
    conversion_result = subprocess.run(
        ["python", "llama.cpp/convert.py", model_directory, "--outfile", gguf_outfile_name, "--outtype", "f16"],
        capture_output=True, text=True
    )

    # Output results for debugging
    print(conversion_result.stdout)
    print(conversion_result.stderr, file=sys.stderr)

    return gguf_outfile_name if os.path.isfile(gguf_outfile_name) else None

# Function to upload the GGUF file to Hugging Face
def upload_gguf_to_hf(gguf_file, model_name, token):
    api = HfApi(token=token)

    if not os.path.isfile(gguf_file):
        raise FileNotFoundError(f"The file {gguf_file} does not exist.")

    api.create_repo(model_name, exist_ok=True, repo_type="model")
    api.upload_file(repo_id=model_name, path_or_fileobj=gguf_file, path_in_repo=os.path.basename(gguf_file))

# Executing the conversion and upload process
model_directory = download_model_from_hf(hf_model_id, model_save_directory)
gguf_file = convert_to_gguf(hf_model_id.split('/')[-1], model_directory)  # Extract the model name

if gguf_file:
    upload_gguf_to_hf(gguf_file, hf_model_id.split('/')[-1], hf_api_token)
    print("GGUF file uploaded to Hugging Face.")
else:
    print("GGUF file conversion failed.")


---
# [Experimental] Generate Conversation Tree V2 (`generate-conversation-tree-async-v2.py`)
---

**Author**: Elijah Kulpinski <br>
**Date**: 12/04/23 <br>
**Version**: 2.1.3 <br>

## Description

This script, updated to version 2.1.0, refines the conversation tree generation process for educational dialogues. Enhancements include deduplication logic, improved parsing and error handling, detailed quality check logging, and various other refinements to ensure high-quality, diverse datasets for LLM fine-tuning.

## Key Features

- **Asynchronous Processing:** Manages concurrent tasks to efficiently generate conversation trees.
- **Deduplication:** Prevents the creation of duplicate conversation entries.
- **Enhanced Parsing:** Robust handling of API responses to ensure valid JSON formatting.
- **Detailed Quality Check Feedback:** Provides specific reasons for quality check failures to aid in debugging.
- **Improved Backoff Mechanism:** Strategically retries after recoverable errors, including rate limits.
- **Finer-Grained Error Handling:** Allows precise tracking of issues at the paragraph level.
- **Enhanced Logging:** Offers comprehensive logging for better traceability and debugging.

## Version History

- **2.1.1:** Moved conversation quality check to using OpenAI's API for consistency, quality, and performance.
- **2.1.0:** Added deduplication, enhanced parsing logic, more advanced quality check NLP checks, and improved error handling.
- **2.0.0:** Introduced a granular rating system, sophisticated rehaul of the branching logic, automated data quality checks, and cost-efficient API utilization strategies.
- **1.3.0:** Implemented log file generation and console output for improved monitoring and debugging. Conducted testing with 'output-snippet.xml' and processed the resulting 'conversation_trees.jsonl' file for data integrity.
- **1.2.0:** Introduced dual-branching logic to simulate 'good' and 'bad' educational responses, creating a dynamic dataset with varied teaching methodologies.
- **1.1.0:** Transitioned from synchronous to asynchronous processing, allowing for parallel generation of conversation trees and optimizing resource usage. Time tracking added to monitor task times for testing.
- **1.0.0:** Initial release. Generated simple, linear conversation trees in a synchronous manner. Basic API rate limit management and conversation context derived directly from textbook content.

## Usage

Run with Python 3.x, ensuring all dependencies from 'requirements.txt' are installed and API keys are set in '.env'. The script processes paragraphs from an XML file and outputs conversation trees to a '.jsonl' file.


In [ ]:
# ==========================================================
#    Functions for Synthetic Conversation Tree Generation
# ==========================================================

# Load the spaCy model for NLP tasks
# nlp = spacy.load("en_core_web_sm") # Smaller model for testing, reduced NLP accuracy
# nlp = spacy.load("en_core_web_lg") # Larger model for deployment, improved NLP accuracy

# Ensure output and log files exist, otherwise create them
if not os.path.exists(INDIVIDUAL_PAIRS_FILE):
    open(INDIVIDUAL_PAIRS_FILE, 'w').close()

if not os.path.exists(RATED_TREES_FILE):
    open(RATED_TREES_FILE, 'w').close()

if not os.path.exists(CONSOLE_LOG_FILE_PATH):
    open(CONSOLE_LOG_FILE_PATH, 'w').close()

# Utility function to log messages to both console and text file
def display_and_log(message):
    print(message)
    with open(CONSOLE_LOG_FILE_PATH, 'a', encoding='utf-8') as log_file:
        log_file.write(f"{message}\n")

# Utility function to parse the XML file and extract paragraphs
def parse_xml(file_path):
    book_paragraphs = []
    try:
        parser = ET.XMLParser(encoding="utf-8")
        tree = ET.parse(file_path, parser=parser)
        root = tree.getroot()
        for book in root.findall('.//book'):
            book_title = book.get('name')
            for paragraph in book.find('.//content').findall('paragraph'):
                text = paragraph.text if paragraph.text is not None else ''
                book_paragraphs.append((book_title, text.strip()))
    except Exception as e:
        display_and_log(f"Error parsing XML file at {file_path}: {e}")
    return book_paragraphs

# Utility function to append data to a file
def append_to_file(file_path, data):
    with open(file_path, 'a', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')

# Backoff handler in case of rate limit error
async def backoff_hdlr(attempt):
    jitter = random.uniform(0, attempt)
    new_delay = min(2 ** attempt, MAX_DELAY) + jitter
    display_and_log(f"Rate limit hit, backing off for {new_delay} seconds.")
    await asyncio.sleep(new_delay) # Puts a single task to sleep not the entire program

# Function to generate a rating by querying the API to evaluate and compare conversations within a tree
async def generate_rating(conversation_tree, client):
    num_conversations = len(conversation_tree["conversation"])
    # Generate a unique identifier for each conversation to include in the prompt
    conversation_identifiers = [f"Conversation {i+1}" for i in range(num_conversations)]

    # Define the prompt to send to the API for rating the conversation
    rating_prompt = (f"Review the following conversations between a student and an educator. "
                   f"Compare each conversation against the others and rank them from 1 (most effective) to "
                   f"{num_conversations} (least effective) in terms of educational quality, considering factors "
                   f"such as clarity, engagement, and the promotion of critical thinking. "
                   f"Respond with your rankings in a list format, preceded by the conversation identifier, "
                   f"For Example: \"Conversation 1: 3, Conversation 2: 1, ..., Conversation {num_conversations}: 2\"."
    )

    # Append a serialized version of each conversation with its identifier
    for idx, identifier in enumerate(conversation_identifiers):
        conversation = json.dumps(conversation_tree["conversation"][idx], ensure_ascii=False)
        rating_prompt += f"\n\n{identifier}: {conversation}"

    # Make the API
    try:
        response = await client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": rating_prompt}]
        )

        # Parse the API response to extract the rankings
        display_and_log(f"Rankings API Response: {response}")
        rankings_response = response.choices[0].message.content.text.strip().split(',')
        display_and_log(f"Rankings Contents: {rankings_response}")
        rankings = {}
        for ranking_pair in rankings_response:
            identifier, rank = ranking_pair.split(':')
            # Extract the number from the identifier (e.g., "Conversation 1" -> 1)
            conv_num = int(identifier.strip().split(' ')[1])
            rankings[conv_num] = int(rank.strip())
            display_and_log(f"Processed Ranking: {conv_num} -> {rank}")

        # Assign the rankings back to the conversation tree using the identifiers
        for index in range(num_conversations):
            conversation_tree["conversation"][index]["ranking"] = rankings[index + 1]
            display_and_log(f"Assigned Ranking to Conversation {index + 1}: {rankings[index + 1]}")

    except Exception as e:
        # Log error and handle it appropriately
        display_and_log(f"Error during rating generation: {e}")

    return conversation_tree  # Return the conversation tree with the rankings

# Function to create a conversation prompt including context management and rating instructions
async def create_conversation_prompt(book_title, paragraph, conversation_history, depth, client):
    summarized_context = await summarize_conversation(conversation_history, client) if conversation_history else ""

    # Conversation start. Root Node.
    if depth == 0:
        prompt = (
        f"Assume the role of an educator assisting a student. You are here to help them develop critical thinking skills and engage with the content."
        f"Based on the concept in the provided paragraph from the book titled: '{book_title}', "
        f"generate a prompt-response pair in JSON for use in a conversation tree in a dataset to fine-tune an LLM to be an educational assistant that "
        f"promote student critical thinking and engagement. Begin from the student's point-of-view with a question reflecting a student's inquiry or "
        f"assigned task regarding the textbook paragraph contents, the student may choose to provide part of an exerpt of the assigned paragraph or try to describe "
        f"the paragraph in their own words as context for the LLM before asking their question, students also don't always use perfect grammar, then follow up from the educator's point-of-view with a response that encourages critical thinking "
        f"and that engages the student. Your response should offer guidance, examples, or suggestions, helping the student explore the concept "
        f"further, while maintaining a professional tone. Avoid directly solving the problem; instead, focus on guiding the student to find the "
        f"solution themselves. If the conversation reaches a natural conclusion or if the topic changes significantly, include "
        f"'[CONVERSATION_END]' or '[TOPIC_END]' respectively.\nHere's the textbook paragraph that was assigned to the student: {paragraph[:500]}"
        f"Don't forget you are to generate both a 'Prompt' and a 'Response' in JSON to represent the start of the conversation between the student and educator."
        f"Example: You receive an assigned textbook paragraph from a math textbook on solving basic integrals for calculus 1 students. "
        f"Example Output: \"Prompt\": \"I just learned that integrals are the opposite of derivatives. Complete this integral: 2x^2 + 2.\", \"Response\": \"I can't solve it for you but I can help guide you towards the solution"
        f"in solving it for yourself. How many steps have you gotten in solving the integral so far? Since integrals are the opposite of derivatives, what are the steps for solving a derivative? "
        f"(Hint: Start by trying to break up this one big integral into multiple smaller, easier-to-solve integrals)\". "
        )

    # Conversation ending. Child nodes.
    elif depth >= MAX_DEPTH - 1 and depth < MAX_DEPTH + 1:
        prompt = (
        f"Assume the role of an educator assisting a student, and are currently in conversation with the student. "
        f"Based on the concept in the provided paragraph from the book titled: '{book_title}', your goal is to start concluding the "
        f"conversation effectively without directly completing the students assignment so they can develop critical thinking skills "
        f"and engage with their course content. Your response should offer guidance, examples, or suggestions, helping the student "
        f"explore the concept further, while maintaining a professional tone. Avoid directly solving the problem; instead, focus on "
        f"guiding the student to find the solution themselves. Your job is to craft a prompt-response pair in JSON, aiming to gracefully "
        f"conclude the interaction. Ensure your response helps in summarizing key points or providing final thoughts, without introducing "
        f"new complex topics. Include '[CONVERSATION_END]' or '[TOPIC_END]' if appropriate. "
        f"\nAssigned Textbook Paragraph Contents: {paragraph[:500]}\nPrior Conversation Summary: {summarized_context}\n"
        f"Don't forget you are to generate both a Prompt and a Response in JSON to represent the conversation approaching its conclusion between the student and educator."
        f"Example 1: \"Prompt\": \"Wow, thanks! I'm feeling more confident for my exam!', \"Response\": \"You're welcome, the key is breaking down complex integrals into simpler parts. Always feel free to revisit our discussion on this topic. Good luck on your exam! [CONVERSATION_END]\""
        f"Example 2: \"Prompt\": \"I'm pretty confident on integrals, time to start English homework, can you assist with that?, \"Response\": \"I'm glad, if you ever want more practice on integrals just ask away. Let me know if I can guide you for any other school content. [TOPIC_END]\""
        )

    # Conversation middle. Sub parent nodes.
    else:
        prompt = (
        f"Assume the role of an educator assisting a student, and are currently in conversation with the student. "
        f"Based on the concept in the provided paragraph from the book titled: '{book_title}', your goal is to continue guiding the student towards learning "
        f"the content they need to answer their question in their inital prompt; without directly completing the students assignment outright, in order for "
        f"them to develop critical thinking skills and engage with their course content. Your response should offer guidance, examples, "
        f"or suggestions, helping the student explore the concept further, while maintaining a professional tone. Avoid directly solving "
        f"the problem; instead, focus on guiding the student to find the solution themselves. Your job is to craft a prompt-response pair "
        f"in JSON. If the conversation reaches a natural conclusion or if the topic changes significantly, include '[CONVERSATION_END]' or '[TOPIC_END]' respectively. "
        f"Assigned Textbook Paragraph Content: {paragraph[:500]}\nPrior Conversation Summary: {summarized_context}\n"
        f"Don't forget you are to generate both a Prompt and a Response in JSON to represent the progression of the conversation between the student and educator."
        f"Example 1: \"Prompt\": \"I never really understood how variables work in programming?\", \"Response\": \"Think of variables as containers for storing data. What data types do you think can be stored in these containers?\""
        f"Example 2: \"Prompt\": \"I don't really know the difference between Comparator and Comparable in Java?\", \"Response\": \"To understand the differences between Comparator and Comparable in Java, consider the purpose and "
        f"usage of each: Comparable is for natural ordering within a class (using compareTo), whereas Comparator allows for custom ordering outside a class (using compare). Reflect on their design implications, "
        f"flexibility, and specific use cases to grasp why and when each interface is used in Java programming.\""
        )
    return prompt

async def summarize_conversation(conversation_history, client):
    # Combine the conversation history into a single string.
    full_history = " ".join(conversation_history)

    # Create the prompt for summarization.
    summary_prompt = (
            f"Please provide a concise summary of the educational interaction below, focusing on key points and main ideas. "
            f"Ensure the summary is coherent and captures the essence of the dialogue without unnecessary details. "
            f"Example Summary: 'The student struggles with the concept of recursion. The educator guides the student by "
            f"comparing recursion to a loop and suggestes practice problems to reinforce learning including The Tower of Hanoi.'"
            f"This summary wil be used as context for prompting a large language model so make sure it's concise while comprehensive "
            f"enough for the LLM to understand the prior conversation between the student and educator.\n\n"
            f"Full Conversation: {full_history}"
        )

    # Make the API call for summarization.
    try:
        response = await client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": summary_prompt}]
        )
        # Assuming the API returns a text response with the summary.
        display_and_log(f"API Summary Response: {response}.")
        summary = response.choices[0].message.content
        display_and_log(f"API Summary: {summary}")
        return summary
    except Exception as e:
        # Handle exceptions and possibly perform a retry or log the error.
        display_and_log(f"Error during API call for summarization: {e}")
        return ""  # Return an empty string as a fallback in case of an error.

def get_value_by_key_variants(dictionary, key_variants_dict, default=None):
    """
    Returns the value for the first key variant found in the dictionary.
    """
    result = {}
    for key, variants in key_variants_dict.items():
        for variant in variants:
            if variant in dictionary:
                result[key] = dictionary[variant]
                break
        if key not in result:
            result[key] = default
    return result

def complete_json_string(json_string, expected_structure):
    try:
        # Attempt to parse the JSON string
        json_obj = json.loads(json_string)

        # Check if the JSON object contains the expected keys
        for key, variants in expected_structure.items():
            if not any(variant in json_obj for variant in variants):
                raise json.JSONDecodeError(f"Missing '{key}' key", json_string, 0)

        return json.dumps(json_obj)

    except json.JSONDecodeError as e:
        try:
            # Check for invalid control characters and remove them
            display_and_log(f"JSONDecodeError: {e}")
            json_string = ''.join(char for char in json_string if (0x20 <= ord(char) <= 0x10FFFF) and char not in ('"', '\\', '\x08', '\x0c', '\x0a', '\x0d', '\x09', '\x00'))

            # Iterate through expected_structure to format JSON
            formatted_json_string = '{'
            for key, variants in expected_structure.items():
                for variant in variants:
                    # Use regex to find each variant in the JSON string
                    match = re.search(fr'"{variant}":"(.*?)"', json_string)
                    if match:
                        formatted_json_string += f'"{key}": "{match.group(1)}", '

            # Remove the trailing comma and space, if any
            if formatted_json_string.endswith(', '):
                formatted_json_string = formatted_json_string[:-2]

            formatted_json_string += '}'

            if formatted_json_string == '{}':  # If no matches were found
                raise json.JSONDecodeError("Unable to format the string correctly", json_string, 0)

            return formatted_json_string

        except json.JSONDecodeError:
            # If it's still not valid JSON, wrap it in an object
            display_and_log(f"JSONDecodeError: {e}")
            return f'{{"data": "{json_string}"}}'

def is_valid_json(json_string):
    try:
        json.loads(json_string)
        return True
    except json.JSONDecodeError:
        return False

def parse_response(response):
    try:
        # Defining the different potential API response types
        expected_structure = {
            "Prompt": ['Prompt', 'prompt', 'PROMPT', 'student', 'student_question', 'question'],
            "Response": ['Response', 'response', 'RESPONSE', 'educator', 'educator_response']
        }

        # Extracting the content of the assistant's message from the response
        display_and_log(f"API Response for Parsing: {response}")
        api_message = response.choices[0].message.content
        display_and_log(f"API Message Extracted: {api_message}")

        # Complete the JSON string if necessary
        if is_valid_json(api_message):
            complete_api_message = api_message
        else:
            display_and_log("Invalid JSON format")
            complete_api_message = complete_json_string(api_message, expected_structure)
            display_and_log(f"Completed JSON API Message: {complete_api_message}")

        # Parse the JSON string
        conversation = json.loads(complete_api_message)

        # Extracting 'Prompt' and 'Response', considering various capitalizations and variants
        parsed_values = get_value_by_key_variants(conversation, expected_structure, default="FAILED_TO_PARSE")
        student_question = parsed_values.get("Prompt", "FAILED_TO_PARSE")
        educator_response = parsed_values.get("Response", "FAILED_TO_PARSE")

        if student_question == "FAILED_TO_PARSE" or educator_response == "FAILED_TO_PARSE":
            raise ValueError("Failed to parse the conversation from the response.")
        else:
            display_and_log(f"Student Question: {student_question} and Educator Response: {educator_response}")

        return student_question, educator_response
    except Exception as e:
        display_and_log(f"Error parsing response: {e}\n")
        return "FAILED_TO_PARSE", "FAILED_TO_PARSE"

# # Function to perform automated checks on the generated conversation locally
# def perform_quality_checks_local(conversation_entry):
#     # Extract educator response and perform NLP analysis
#     response = conversation_entry['educator_response']
#     doc = nlp(response)

#     # Check for educational relevance: Presence of explanations, definitions, or clarifications
#     educational_relevance = any(token.pos_ in ["VERB", "NOUN"] and token.dep_ in ["ROOT", "attr", "dobj"] for token in doc)

#     # Sentiment analysis: Check for neutral to positive tone, as educational content should be informative and encouraging
#     sentiment = TextBlob(response).sentiment.polarity
#     positive_sentiment = sentiment >= -0.1  # Allowing slightly negative sentiment for challenging topics

#     # Contextual understanding: Analyze for cohesive and meaningful sentences
#     contextual_understanding = any(sent.root.pos_ == "VERB" and sent.root.dep_ == "ROOT" for sent in doc.sents)

#     # Length and grammar check: Assuming longer and grammatically correct responses may have more explanations/details
#     length_check = len(response.split()) > 20  # Arbitrary length threshold
#     grammar_check = any(punctuation in response for punctuation in ['.', '?', '!'])

#     # All checks must pass for the conversation entry to be considered high quality
#     quality_checks_passed = all([
#         educational_relevance,
#         positive_sentiment,
#         contextual_understanding,
#         length_check,
#         grammar_check
#     ])

#     return quality_checks_passed

# Function to perform automated checks on the generated conversation using OpenAIs API
async def perform_quality_checks_api(conversation_entry, client):
    # Create a prompt for the API to evaluate the conversation entry
    evaluation_prompt = (
        f"Please evaluate the educational quality of the following conversation "
        f"between a student and an educator. Assess it for clarity, engagement, "
        f"and promotion of critical thinking. Respond with an overall score from "
        f"1 (no conversation/unintelligible) to 50 (unhelpful/incorrect) to 100 "
        f"(perfect interaction) in JSON format with the label \'Score\'. If you "
        f"fail to respond in the expected JSON (\"Score\":\"#\"), the assumed score is 0."
        f"Note: Failed to Parse should recieve an automatic 0 since it isn't a conversation, "
        f"but rather an issue with this scripts logic. Otherwise, grade as defined above.\n\n"
        f"Student Question: {conversation_entry['student_question']}\n"
        f"Educator Response: {conversation_entry['educator_response']}\n"
        f"Example Response: \"Score\":\"73\""
    )

    # For score format
    expected_structure = {
        "Score": ['Score', 'score', 'SCORE']
    }

    # Make the API call for evaluation
    try:
        response = await client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": evaluation_prompt}]
        )
        # Check if 'Score' is in the response
        if "Score" in response.choices[0].message.content:
            api_response = complete_json_string(response.choices[0].message.content, expected_structure)
            api_response = json.loads(api_response)
            score = int(api_response["Score"])
            display_and_log(f"Received Score: {score}")
            return score >= 60  # A score of 60+ is passing
        else:
            display_and_log("Score key not found in API response")
            return False
    except Exception as e:
        display_and_log(f"Error during API call for quality evaluation: {e}")
        return False  # Treat errors as a failure in quality check

# Helper function to generate a hash of the conversation content
def hash_conversation(conversation):
    # Create a unique string representation of the conversation
    conversation_str = json.dumps(conversation, sort_keys=True)
    # Use SHA-256 hash function to generate a hash
    return hashlib.sha256(conversation_str.encode('utf-8')).hexdigest()

# Function to check for duplicate conversations using hashing
def deduplicate_conversation(new_entry, conversation_history_hashes):
    # Generate the hash for the new entry
    new_entry_hash = hash_conversation(new_entry)

    # Check if the hash of the new entry is in the set of hashes
    if new_entry_hash in conversation_history_hashes:
        return True  # Duplicate found

    # Add the new entry hash to the history set for future checks
    conversation_history_hashes.add(new_entry_hash)
    return False  # No duplicate found

# Async function to generate a single branch of a conversation tree
async def generate_conversation_branch(client, book_title, paragraph, semaphore, index, current_depth=0):
    async with semaphore:  # Control concurrency with semaphore
        display_and_log(f"Starting branch generation at depth {current_depth} for paragraph {index} ('{book_title}').")

        if current_depth >= MAX_DEPTH - 1:
            return None

        conversation_branch = {
            "book_title": book_title,
            "paragraph_index": index,
            "conversation": [],
            "depth": current_depth
        }
        conversation_history = []  # Maintain conversation history for context
        conversation_history_hashes = set()  # Set to store hashes of unique conversation entries

        try:
            prompt = await create_conversation_prompt(book_title, paragraph, conversation_history, current_depth, client)
            start_time = time.perf_counter()
            display_and_log(f"Paragraph {index} ('{book_title}'): Generating Prompt-Response (Depth: {current_depth}). Start Time: {start_time:.2f}")

            response = await client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": prompt}]
            )
            parsed_response = parse_response(response)
            display_and_log(f"Parsed Response: {parsed_response}.")
            student_question, educator_response = parsed_response
            display_and_log(f"Student Question: {student_question} - Educator Response: {educator_response}")

            entry = {
                "book_title": book_title,
                "paragraph_index": index,
                "depth": current_depth,
                "student_question": student_question,
                "educator_response": educator_response
            }

            if not deduplicate_conversation(entry, conversation_history_hashes):
                if await perform_quality_checks_api(entry, client):
                    conversation_branch["conversation"].append(entry)
                    append_to_file(INDIVIDUAL_PAIRS_FILE, entry)
                    conversation_history.append(student_question + " " + educator_response)

                    if "[CONVERSATION_END]" and "[TOPIC_END]" not in educator_response and current_depth < MAX_DEPTH - 1:
                        deeper_paragraph = educator_response
                        deeper_branch = await generate_conversation_branch(client, book_title, deeper_paragraph, semaphore, index, current_depth + 1)
                        if deeper_branch and deeper_branch["conversation"]:
                            conversation_branch["conversation"].extend(deeper_branch["conversation"])
            else:
                display_and_log(f"Branch at depth {current_depth} for paragraph {index} ('{book_title}') failed checks or is a duplicate.")

        except Exception as e:
            display_and_log(f"Error in generate_conversation_branch: {e}")

        end_time = time.perf_counter()
        display_and_log(f"Finished processing branch at depth {current_depth} for paragraph {index} ('{book_title}') in total time: {end_time - start_time:.2f} seconds.")
        return conversation_branch

# Async function to orchestrate the generation of a full conversation tree
async def generate_conversation_tree(client, semaphore, book_title, paragraph, index):
    try:
        start_time = time.perf_counter()
        root_branch = await generate_conversation_branch(client, book_title, paragraph, semaphore, index)

        if root_branch is None or not root_branch["conversation"]:
            display_and_log(f"No valid conversation generated for paragraph {index} ('{book_title}').")
            return None

        # Temporary storage for the conversation tree before rating
        temp_conversation_tree = {"branches": [root_branch]}

        # Before calling generate_rating, add a check for 'conversation' key
        if "conversation" not in temp_conversation_tree:
            display_and_log("No 'conversation' key found in temp_conversation_tree")
            return None

        # Generate a rating for the conversation tree
        rated_tree = await generate_rating(temp_conversation_tree, client)
        if rated_tree:
            # Append the rated tree to the file only after successful rating
            append_to_file(RATED_TREES_FILE, rated_tree)

        end_time = time.perf_counter()
        display_and_log(f"Generated full conversation tree for paragraph {index} ('{book_title}') in total time: {end_time - start_time:.2f} seconds.")
        return rated_tree

    except Exception as e:
        display_and_log(f"Error in generate_conversation_tree for Paragraph {index}: {e}")
        return None

# Async main function to process all paragraphs and generate conversation trees
async def process_paragraphs_async(book_paragraphs):
    async_client = AsyncOpenAI(api_key=API_KEY)
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_TASKS)
    tasks = []

    try:
        for index, (book_title, paragraph) in enumerate(book_paragraphs):
            display_and_log(f"Reading paragraph {index + 1}/{len(book_paragraphs)} from the book '{book_title}'.")
            task = generate_conversation_tree(async_client, semaphore, book_title, paragraph, index)
            tasks.append(task)

        trees = await asyncio.gather(*tasks, return_exceptions=True)

        for tree in trees:
            if isinstance(tree, Exception):
                display_and_log(f"Exception during task execution: {tree}")

    except Exception as e:
        display_and_log(f"Error processing paragraphs: {e}\n")

    finally:
        # Delay added to stay well under the OpenAI API rate limit.
        await asyncio.sleep(0.5)  # Throttling to stay within rate limits

# Entry point of the script
display_and_log(f"Starting script. Querying OpenAI's API with {XML_FILE_PATH} contents to {INDIVIDUAL_PAIRS_FILE} and {RATED_TREES_FILE} in the same directory.")
book_paragraphs = parse_xml(XML_FILE_PATH)
asyncio.run(process_paragraphs_async(book_paragraphs))
display_and_log(f"The synthetic dataset has finished generating.")


---

### Conclusion and Future Work

---

#### **Project Overview**

This notebook consolidates several scripts into a comprehensive, reproducible workflow for enhancing language models with educational applications. It provides a step-by-step guide for those interested in replicating or extending our research, integrating various components into a unified Python notebook.

#### **Key Achievements**

- **Dataset Generation and Processing:** Conversion of textbook content into XML and conversation trees, creating training data for educational language models.
- **Fine-Tuning of Language Models:** Using Mistral-OpenOrca for fine-tuning, aligning the model closely with educational content.
- **Model Conversion and Optimization:** Conversion to GGUF format for efficient CPU inference, facilitating deployment across various platforms.

#### **Implications and Applications**

- **Educational Assistant Development:** The project's outcomes are key to building AI-driven educational assistants, enhancing personalized learning experiences.
- **Research Extension Opportunities:** This foundation enables further exploration in different domains, offering diverse experimental possibilities.

#### **Future Work Recommendations**

- **Educational Context Expansion:** Adapt the model to a broader range of educational settings and subjects to enhance its applicability.
- **Vectorized Databases Integration:** Implement vectorized databases to remember student work and learning styles, personalizing the educational experience.
- **Mixture of Experts (MoE) Utilization:** Employ MoE models to improve response accuracy, reduce hallucinations, distribute computational load, and allow multiple models to contribute to the system.
- **LLM Wrapper Incorporation:** Integrate LLM Wrappers like LangChain to provide access to Learning Management Systems (LMS) such as Canvas, Blackboard, or D2L, enhancing the model's utility in educational environments.
- **Enhanced User Interaction:** Focus on improving the interface and interaction mechanisms of AI-driven educational tools for a more engaging learning experience.
- **Scalability and Efficiency:** Explore methods to scale the model efficiently without compromising performance.
- **Real-time Interaction Capabilities:** Enhance the model to support real-time interactions for dynamic educational contexts.
- **Interdisciplinary Applications:** Expand the model's use to interdisciplinary studies to explore new areas in AI-driven education.

#### **Final Thoughts**

This notebook is a step forward in AI's application in education and an invitation for the community to contribute to this evolving field. The opportunities for innovation are vast, and diverse perspectives will play a crucial role in advancing educational technology.
